In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_= pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Trip_leg.xlsx')
dv = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Household.xlsx')
profile = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Profile.xlsx')
survey = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Survey.xlsx')

In [3]:
profile

,p_id,hh_id,participation_status,person_order,name,age,gender,District,edu,drive_lic,...,sector,job,work_time,office_info,work_days,pt_card,pt_card_type,disability,had_trip,no_trip_reason
0,71,6kaqzPZwvLgUnuUalhTx,Evet,K1,Şükran Çiftçi,64.0,Kadın,BEYKOZ,İlköğretim mezunu,Yok,...,NaN,NaN,NaN,NaN,0.0,Yok,NaN,Hayır,Hayır,Gerek Olmaması
1,72,6kaqzPZwvLgUnuUalhTx,Evet,K2,Mahmut,67.0,Erkek,BEYKOZ,İlköğretim mezunu,Yok,...,NaN,NaN,NaN,NaN,0.0,Var,Ücretsiz Kart,Hayır,Evet,NaN
2,73,6kaqzPZwvLgUnuUalhTx,Evet,K3,Serkan,42.0,Erkek,BEYKOZ,Lise/Dengi mezun,Var,...,Diğer,Diğer,Tam zamanlı,Sabit İş yeri/Ofis,5.0,Var,Anonim Kart(Tam),Hayır,Hayır,Gerek Olmaması
3,74,6kaqzPZwvLgUnuUalhTx,Evet,K4,Serdar,40.0,Erkek,BEYKOZ,Lise/Dengi mezun,Var,...,Diğer,Diğer,Tam zamanlı,Birden fazla İş yeri/Ofis,5.0,Var,Anonim Kart(Tam),Hayır,Hayır,Gerek Olmaması
4,75,6v8e5FWFp6A0rG05P5U9,Evet,K1,şerife acet,55.0,Kadın,ESENLER,İlköğretim mezunu,Yok,...,NaN,NaN,NaN,NaN,NaN,Var,Anonim Kart(Tam),Hayır,Evet,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169418,195249,2ZSpVEcgNCwGRkKoG8Uz,Uygun Değil,K3,Özüm naz ovacik,5.0,Kadın,ŞİLE,Okula başlamamış,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hayır,Uygun Değil,Uygun Değil
169419,195250,VT1bA6J3NwEbTafh7O07,Evet,K1,Ayşe hanım,40.0,Kadın,ŞİLE,Lise/Dengi mezun,Var,...,NaN,NaN,NaN,NaN,0.0,Var,Anonim Kart(Tam),Hayır,Hayır,Gerek Olmaması
169420,195251,VT1bA6J3NwEbTafh7O07,Evet,K2,Murat bey,45.0,Erkek,ŞİLE,Lise/Dengi mezun,Var,...,Gıda,Aşçı,Tam zamanlı,Sabit İş yeri/Ofis,7.0,Var,Anonim Kart(Tam),Hayır,Evet,NaN
169421,195252,XK9uBTlpBSu0g69MTu6G,Evet,K1,Turgut bey,40.0,Erkek,ŞİLE,Lise/Dengi mezun,Var,...,Otomotiv,Diğer,Tam zamanlı,Sabit İş yeri/Ofis,6.0,Var,Anonim Kart(Tam),Hayır,Evet,NaN


### 🧩 DATA PROCESS - Employment Group Kolonunun Oluşturulması ve Trip Tablosuna Eklenmesi
## 1️⃣ Employment Group Kolonunun Oluşturulması (`profile` tablosunda)

Aşağıdaki kurallara göre `profile` tablosunda yeni bir `Employment Group` sütunu oluşturulmuştur:

- **Full Time**: 18 yaş üstü, tam zamanlı çalışan ve öğrenci olmayan bireyler
- **Part Time**: 18 yaş üstü, yarı zamanlı çalışan ve öğrenci olmayan bireyler
- **Adult Student**: 18 yaş üstü ve hâlen öğrenci olan bireyler
- **Adult Unemployed**: 18-65 yaş arası, çalışmayan bireyler
- **65+ Retired**: 65 yaş üstü ve çalışmayan bireyler
- **Child Student**: 6-18 yaş arası ve öğrenci olan bireyler
- **Child**: 6-18 yaş arası ve öğrenci olmayan bireyler
- **Other**: Yukarıdaki kategorilere girmeyen bireyler

In [4]:
import pandas as pd
import numpy as np
import re

# Örnek olarak profile ve df_ veri çerçevelerinizin zaten yüklü olduğunu varsayıyorum.

# 'öğrenci' ifadesini içeren satırları bulmak için yardımcı değişken
is_student = profile['edu'].str.contains('öğrenci', case=False, na=False)

# Koşullar (SQL'deki sıralamaya sadık)
conditions = [
    # 1. Full Time
    (profile['work_status'] == 'Çalışıyor') &
    (profile['age'] >= 18) &
    (profile['work_time'] == 'Tam zamanlı') &
    (~is_student),

    # 2. Part Time
    (profile['work_status'] == 'Çalışıyor') &
    (profile['age'] >= 18) &
    (profile['work_time'] == 'Yarı zamanlı') &
    (~is_student),

    # 3. Adult Student
    (profile['age'] >= 18) & is_student,

    # 4. Adult Unemployed
    (profile['work_status'] == 'Çalışmıyor') &
    (profile['age'].between(18, 65)),

    # 5. 65+ Retired
    (profile['work_status'] == 'Çalışmıyor') &
    (profile['age'] >= 65),

    # 6. Child Student
    (profile['age'].between(6, 18)) & is_student,

    # 7. Child
    (profile['age'].between(6, 18)) & (~is_student)
]

# Bu koşullara karşılık gelen etiketler
choices = [
    'Full Time',
    'Part Time',
    'Adult Student',
    'Adult Unemployed',
    '65+ Retired',
    'Child Student',
    'Child'
]

# Koşul/etiket listelerini tek satırda profile_group kolonuna dönüştür
profile['Employment Group'] = np.select(conditions, choices, default='Other')

# hh_id ve person_order'ı birleştirerek yeni bir kolon oluştur
profile['person_id'] = profile['hh_id'].astype(str) + '-' + profile['person_order'].astype(str)
print(profile.head(2))

missing_count = profile['Employment Group'].isna().sum()
print(f"Employment_Group sütununda {missing_count} adet eksik veri bulunmaktadır.")

# --- Burada t_leg_prefix düzgün şekilde regex ile çıkarılacak ---

# t_leg_id'den prefix'i K ve ardından gelen rakam(lar) olarak çekmek için regex kullan
df_['t_leg_prefix'] = df_['t_leg_id'].astype(str).str.extract(r'(K\d+)')

# hh_id ve person_order (burada t_leg_prefix) birleştirerek person_id oluştur
df_['person_id'] = df_['hh_id'].astype(str) + '-' + df_['t_leg_prefix'].astype(str)

# 1. Survey tablosunda 'Görüşme Tamamlandı' olan id'leri al
completed_ids = survey[survey['Result'] == 'Görüşme Tamamlandı']['ID'].unique()

# 2. df_ içinde sadece bu id'lerle eşleşen hh_id'lere sahip satırları filtrele
df_ = df_[df_['hh_id'].isin(completed_ids)]

# profile'dan Employment Group ve gender sütunlarını df_'ye merge et
df_ = df_.merge(profile[['person_id', 'Employment Group', 'gender','edu']], on='person_id', how='left')

# Sonuç kontrolü
df_


   p_id                 hh_id participation_status person_order  \
0    71  6kaqzPZwvLgUnuUalhTx                 Evet           K1   
1    72  6kaqzPZwvLgUnuUalhTx                 Evet           K2   

             name   age gender District                edu drive_lic  ...  \
0  Şükran Çiftçi   64.0  Kadın   BEYKOZ  İlköğretim mezunu       Yok  ...   
1          Mahmut  67.0  Erkek   BEYKOZ  İlköğretim mezunu       Yok  ...   

  work_time office_info work_days pt_card   pt_card_type disability had_trip  \
0       NaN         NaN       0.0     Yok            NaN      Hayır    Hayır   
1       NaN         NaN       0.0     Var  Ücretsiz Kart      Hayır     Evet   

   no_trip_reason  Employment Group                person_id  
0  Gerek Olmaması  Adult Unemployed  6kaqzPZwvLgUnuUalhTx-K1  
1             NaN       65+ Retired  6kaqzPZwvLgUnuUalhTx-K2  

[2 rows x 25 columns]
Employment_Group sütununda 0 adet eksik veri bulunmaktadır.


,hh_id,id,p_id,t_leg_id,purpose,weekly_trip_count,has_transfer,mode,acc_p,pc_reason,...,dest_lat,dest_lon,person_id,t_id,tl_id,p_t_id,t_leg_prefix,Employment Group,gender,edu
0,6kaqzPZwvLgUnuUalhTx,55,72,K2Y1E1,Alışveriş,4.0,Hayır,Yürüme,Tek,NaN,...,0.00000000° N,0.00000000° E,6kaqzPZwvLgUnuUalhTx-K2,Y1,E1,72Y1,K2,65+ Retired,Erkek,İlköğretim mezunu
1,6kaqzPZwvLgUnuUalhTx,56,72,K2Y2E1,Eve gidiş,4.0,Hayır,Yürüme,Tek,NaN,...,41.09473358° N,29.08219721° E,6kaqzPZwvLgUnuUalhTx-K2,Y2,E1,72Y2,K2,65+ Retired,Erkek,İlköğretim mezunu
2,6v8e5FWFp6A0rG05P5U9,57,75,K1Y1E1,Alışveriş,2.0,Hayır,Yürüme,Tek,NaN,...,41.05752300° N,28.86980000° E,6v8e5FWFp6A0rG05P5U9-K1,Y1,E1,75Y1,K1,Adult Unemployed,Kadın,İlköğretim mezunu
3,6v8e5FWFp6A0rG05P5U9,58,75,K1Y2E1,Eve gidiş,2.0,Hayır,Yürüme,Tek,NaN,...,0.00000000° N,0.00000000° E,6v8e5FWFp6A0rG05P5U9-K1,Y2,E1,75Y2,K1,Adult Unemployed,Kadın,İlköğretim mezunu
4,NAVrD2WQ3YeXoDthyEdd,59,76,K1Y1E1,Diğer,1.0,Evet,Belediye Otobüsü,Tek,NaN,...,41.01894170° N,29.05762980° E,NAVrD2WQ3YeXoDthyEdd-K1,Y1,E1,76Y1,K1,Adult Unemployed,Erkek,Üniversite mezun (Ön Lisans/Lisans)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245249,yycdcequtEDveWUJFuKs,2232413,"2,193,816",D1Y2E1,Eve gidiş,5.0,Hayır,Yürüme,NaN,NaN,...,41.03134682° N,28.87287612° E,yycdcequtEDveWUJFuKs-nan,Y2,E1,193815_D1Y2,NaN,NaN,NaN,NaN
245250,DkuNiQYoJeBPg5AYCj6H,2232537,"2,193,965",D3Y1E1,İşe gidiş,6.0,Hayır,Yürüme,NaN,NaN,...,0.00000000° N,0.00000000° E,DkuNiQYoJeBPg5AYCj6H-nan,Y1,E1,193962_D3Y1,NaN,NaN,NaN,NaN
245251,Qj247VNWaMK7v1mFC14Y,2232626,"2,194,067",D4Y1E1,Diğer,5.0,Hayır,Yürüme,NaN,NaN,...,41.03269375° N,28.87494551° E,Qj247VNWaMK7v1mFC14Y-nan,Y1,E1,194063_D4Y1,NaN,NaN,NaN,NaN
245252,KwopuIgNhOqaYGT4F80V,2232685,"2,194,162",D1Y1E1,Sağlık Kurumuna gidiş,3.0,Hayır,Özel Araç,NaN,NaN,...,0.00000000° N,0.00000000° E,KwopuIgNhOqaYGT4F80V-nan,Y1,E1,194161_D1Y1,NaN,NaN,NaN,NaN


###  District  Kolonunun Oluşturulması (`trip` tablosuna eklenmesi)

In [5]:
df_ = df_.merge(dv[['hh_id', 'district']], on='hh_id', how='left')

In [6]:
len(df_)

245254

In [7]:
df_.head(1)

,hh_id,id,p_id,t_leg_id,purpose,weekly_trip_count,has_transfer,mode,acc_p,pc_reason,...,dest_lon,person_id,t_id,tl_id,p_t_id,t_leg_prefix,Employment Group,gender,edu,district
0,6kaqzPZwvLgUnuUalhTx,55,72,K2Y1E1,Alışveriş,4.0,Hayır,Yürüme,Tek,NaN,...,0.00000000° E,6kaqzPZwvLgUnuUalhTx-K2,Y1,E1,72Y1,K2,65+ Retired,Erkek,İlköğretim mezunu,BEYKOZ


### 🧼 İlçe Adı Standartlaştırma

Yolculuk verisindeki bazı ilçe adları farklı şekillerde yazılmış olabilir.  
Özellikle `"EYÜP"` ismi `"EYÜPSULTAN"` olarak güncellenmiştir.

In [8]:
df_["orig_dist"] = df_["orig_dist"].str.upper().replace("EYÜP", "EYÜPSULTAN")
df_["district"] = df_["district"].str.upper().replace("EYÜP", "EYÜPSULTAN")
df_["dest_dist"] = df_["dest_dist"].str.upper().replace("EYÜP", "EYÜPSULTAN")


In [9]:
df=df_

### 🚏 Ulaşım Modu Kategorilendirmesi

Yolculuk verilerindeki `mode` (ulaşım türü) bilgisini daha genel kategorilere ayırmak amacıyla sınıflandırma yapılmıştır:

- **Public Transport**: Metro, otobüs, tramvay, vapur, Marmaray vb. tüm toplu taşıma türleri  
- **Micromobility**: Bisiklet, paylaşımlı bisiklet/skuter gibi bireysel ve kısa mesafeli araçlar  
- **Private Vehicle**: Özel araç, taksi, motosiklet, kurum aracı gibi bireysel motorlu taşıtlar  
- **Service**: Servis araçları  
- **Walking** durumlar

Aşağıdaki Python fonksiyonu ile `mode` sütunundaki her bir değer uygun kategoriye atanmış ve sonuç `mode_category` adlı yeni bir kolona eklenmiştir.


In [10]:
# Kategori kümeleri
public_transport = {
    "Belediye Otobüsü", "Deniz Motoru", "Deniz Otobüsü (IDO)", "Deniz Taksi", "Dolmuş", "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı", "Marmaray", "Metro", "Metrobüs", "Minibüs",
    "Şehir Hatları Vapuru", "Teleferik", "Tramvay", "Tünel"
}
micromobility = {"Bisiklet", "Paylaşımlı Bisiklet", "Skuter", "Paylaşımlı Skuter"}
private_vehicle = {
    "Motosiklet", "Özel Araç", "Paylaşımlı Araç", "Taksi", "Kiralık Araç", "Korsan Taksi", 
    "Kurum/ Firma Aracı", "Diğer"
}
service = {"Servis"}
walking = {"Yürüme"}

# Sınıflandırma fonksiyonu
def classify_mode(mode):
    if mode in public_transport:
        return "Public Transport"
    elif mode in micromobility:
        return "Micromobility"
    elif mode in private_vehicle:
        return "Private Vehicle"
    elif mode in service:
        return "Service"
    elif mode in walking:
        return "Walking"
    else:
        return "Other"

# Yeni kolon olarak ekle
df["mode_category"] = df["mode"].apply(classify_mode)

### 🧾 Analiz İçin Gerekli Kolonların Seçilmesi

Analiz sürecinde yalnızca gerekli olan değişkenlerin kullanılması amacıyla `df` veri çerçevesinden aşağıdaki kolonlar seçilmiştir:

- Hane ve kişi bilgileri: `hh_id`, `p_id`, `id`, `t_leg_id`
- Ulaşım bilgileri: `mode`, `mode_category`, `purpose`, `start_time`, `end_time`, `waiting_time`
- Konum bilgileri: `orig_lat`, `orig_lon`, `dest_lat`, `dest_lon`, `orig_loc_type`, `dest_loc_type`
- İlçe bilgisi: `district`, `orig_dist`, `dest_dist`
- Erişim ve erişim sonrası süreler: `access_time`, `egress_time`

Bu kolonlar `selected_columns` listesiyle belirlenmiş ve `df_selected` adlı yeni bir DataFrame oluşturulmuştur.


In [11]:
# İstenen kolonları tanımla
selected_columns = [
    "hh_id", "id", "p_id","acc_p","person_id","t_leg_id", "purpose", "gender","mode","orig_lat","orig_lon","dest_lat","dest_lon","edu",
    "orig_loc_type", "orig_dist", "access_time", "egress_time","district","mode_category","Employment Group",
    "start_time", "waiting_time", "dest_loc_type", "dest_dist", "end_time",
]

# Belirtilen kolonları içeren yeni DataFrame oluştur
df_selected = df[selected_columns].copy()
df_selected.head(1)

,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,mode,orig_lat,...,access_time,egress_time,district,mode_category,Employment Group,start_time,waiting_time,dest_loc_type,dest_dist,end_time
0,6kaqzPZwvLgUnuUalhTx,55,72,Tek,6kaqzPZwvLgUnuUalhTx-K2,K2Y1E1,Alışveriş,Erkek,Yürüme,41.09473358° N,...,0.0,0.0,BEYKOZ,Walking,65+ Retired,09:20,0.0,DİĞER,BEYKOZ,09:30


In [12]:
len(df_selected)

245254

### 🕒 Başlangıç ve Bitiş Saatlerinde Manuel Düzeltmeler

Veri setinde bazı yolculukların `start_time` ve `end_time` değerlerinde eksiklikler veya tutarsızlıklar tespit edilmiştir.  
Bu nedenle aşağıdaki `id` değerlerine sahip satırlar için manuel zaman düzeltmeleri yapılmıştır:

- Zaman bilgisi eksik olan ya da mantıksal sıra bozukluğu gösteren yolculuklar belirlenmiştir.
- Her `id` için doğru başlangıç (`start_time`) veya bitiş saati (`end_time`) atanmıştır.
- Bu düzeltmeler, analizlerde zaman serisi hesaplamalarının sağlıklı yapılabilmesini sağlar.

Kod bloğu içerisinde `df_selected.loc[...]` kullanılarak doğrudan ilgili satırlara yeni saat değerleri atanmıştır.


In [13]:
df_selected.loc[df_selected["id"] == "117,973", "end_time"] = "09:00"

df_selected.loc[df_selected["id"] == "167,245", "start_time"] = "13:00"
df_selected.loc[df_selected["id"] == "167,245", "end_time"] = "13:30"
df_selected.loc[df_selected["id"] == "167,246", "start_time"] = "13:30"



df_selected.loc[df_selected["id"] == "5,053", "end_time"] = "09:15"
df_selected.loc[df_selected["id"] == "5,054", "end_time"] = "09:45"
df_selected.loc[df_selected["id"] == "5,055", "start_time"] = "09:45"
df_selected.loc[df_selected["id"] == "5,055", "end_time"] = "10:10"


df_selected.loc[df_selected["id"] == "19,714", "end_time"] = "11:00"

df_selected.loc[df_selected["id"] == "40,806", "start_time"] = "13:30"
df_selected.loc[df_selected["id"] == "40,807", "end_time"] = "13:55"


df_selected.loc[df_selected["id"] == "44,618", "start_time"] = "07:00"
df_selected.loc[df_selected["id"] == "44,619", "end_time"] = "09:30"

df_selected.loc[df_selected["id"] == "53,395", "end_time"] = "09:00"

df_selected.loc[df_selected["id"] == "77,259", "end_time"] = "13:50"

df_selected.loc[df_selected["id"] == "87,614", "end_time"] = "11:00"

df_selected.loc[df_selected["id"] == "96,034", "start_time"] = "08:00"

df_selected.loc[df_selected["id"] == "114,721", "end_time"] = "07:45"

df_selected.loc[df_selected["id"] == "224,885", "end_time"] = "11:00"

### ⏱️ Travel Time ve Vehicle Time Hesaplanması

Bu bölümde, her yolculuk için `start_time` ve `end_time` değerlerine göre toplam seyahat süresi hesaplanmıştır:

- `travel_time`: Yolculuğun başlangıç ve bitiş saatleri arasındaki toplam süre (dakika cinsinden)
    - Eğer `end_time` başlangıç saatinden küçükse (örneğin gece yarısını geçtiyse), 1 gün eklenerek hesaplama yapılır.
    - Eksik veya hatalı saat verileri varsa hesaplama dışı bırakılır.
- `vehicle_time`: `travel_time` değerinden şu süreler çıkarılarak hesaplanır:
    - `access_time` (ulaşım noktasına erişim süresi)
    - `waiting_time` (bekleme süresi)
    - `egress_time` (varış noktasından nihai varışa geçiş süresi)

Bu sayede, yolculukların sadece araç içinde geçirilen süresi (`vehicle_time`) de belirlenmiş olur.


In [14]:
# Belirtilen sütunlardaki NaN değerleri 0 ile doldur
df_selected[["access_time", "egress_time", "waiting_time"]] = df_selected[["access_time", "egress_time", "waiting_time"]].fillna(0)

from datetime import datetime, timedelta 
# Süre hesaplama fonksiyonu
def calculate_duration(start_str, end_str):
    time_format = "%H:%M"

    # Eğer veri eksikse None döndür
    if pd.isna(start_str) or pd.isna(end_str):
        return None

    try:
        start = datetime.strptime(str(start_str), time_format)
        end = datetime.strptime(str(end_str), time_format)
    except ValueError:
        return None  # Geçersiz saat formatı varsa atla

    if end < start:
        end += timedelta(days=1)  # Gece yarısı aşımı için

    return (end - start).total_seconds() / 60  # dakika

# Uygula
df_selected["travel_time"] = df_selected.apply(lambda row: calculate_duration(row["start_time"], row["end_time"]), axis=1)
df_selected["vehicle_time"] = df_selected["travel_time"] - df_selected["access_time"] - df_selected["waiting_time"] - df_selected["egress_time"]
df_selected.head(1)

,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,mode,orig_lat,...,district,mode_category,Employment Group,start_time,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time
0,6kaqzPZwvLgUnuUalhTx,55,72,Tek,6kaqzPZwvLgUnuUalhTx-K2,K2Y1E1,Alışveriş,Erkek,Yürüme,41.09473358° N,...,BEYKOZ,Walking,65+ Retired,09:20,0.0,DİĞER,BEYKOZ,09:30,10.0,10.0


In [15]:
len(df_selected)

245254

### 🔗 Yolculuk Gruplarının Etiketlenmesi (`same_p_and_k_y`)

Her bir bireyin (`p_id`) farklı yolculuk serileri (`t_leg_id`) içinde yer alması mümkündür.  
Bu adımda, bireylerin aynı yolculuk kümesine ait olup olmadığını belirlemek için iki aşamalı bir işlem yapılmıştır:

1. **`k_y_part` Oluşturulması**:  
   - `t_leg_id` kolonundaki ifadeler `"KxYxEx"` yapısındadır.
   - Buradan yalnızca `"KxYx"` kısmı alınarak yeni bir `k_y_part` kolonu oluşturulmuştur.

2. **Yolculuk Serisi Etiketleme (`same_p_and_k_y`)**:  
   - Aynı `p_id` ve `k_y_part` değerine sahip satırlar aynı yolculuk grubudur.
   - `groupby(...).ngroup()` fonksiyonu ile her bir yolculuk serisi benzersiz bir sayısal etiket alır.
   - Böylece her bireyin her yolculuk serisi ayrı ayrı takip edilebilir hale gelir.

Sonuç olarak, `same_p_and_k_y` kolonu analizlerde yolculuk bazlı gruplamaları kolaylaştırmak için kullanılır.
apılmıştır.


In [16]:
import pandas as pd

# t_leg_id'nin 'KxYx' kısmını al
df_selected["k_y_part"] = df_selected["t_leg_id"].str.extract(r'^(K\d+Y\d+)')

# p_id + k_y_part kombinasyonuna göre gruplandır ve etiketle
df_selected["same_p_and_k_y"] = df_selected.groupby(["p_id", "k_y_part"]).ngroup()

# same_p_and_k_y kolonundaki tekrar sayımlarını al
group_sizes = df_selected["same_p_and_k_y"].value_counts()

# Her tekrar sayısının kaç grupta görüldüğünü hesapla
repeat_distribution = group_sizes.value_counts().sort_index()

# Sonuçları yazdır
for repeat_count, group_count in repeat_distribution.items():
    print(f"{repeat_count} kere tekrar eden grup sayısı: {group_count}")


1 kere tekrar eden grup sayısı: 230872
2 kere tekrar eden grup sayısı: 4048
3 kere tekrar eden grup sayısı: 557
4 kere tekrar eden grup sayısı: 63


In [17]:
df_selected

,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,mode,orig_lat,...,Employment Group,start_time,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time,k_y_part,same_p_and_k_y
0,6kaqzPZwvLgUnuUalhTx,55,72,Tek,6kaqzPZwvLgUnuUalhTx-K2,K2Y1E1,Alışveriş,Erkek,Yürüme,41.09473358° N,...,65+ Retired,09:20,0.0,DİĞER,BEYKOZ,09:30,10.0,10.0,K2Y1,198914.0
1,6kaqzPZwvLgUnuUalhTx,56,72,Tek,6kaqzPZwvLgUnuUalhTx-K2,K2Y2E1,Eve gidiş,Erkek,Yürüme,0.00000000° N,...,65+ Retired,09:35,0.0,EV(SADECE İKAMET EDİLEN EV),BEYKOZ,09:40,5.0,5.0,K2Y2,198915.0
2,6v8e5FWFp6A0rG05P5U9,57,75,Tek,6v8e5FWFp6A0rG05P5U9-K1,K1Y1E1,Alışveriş,Kadın,Yürüme,0.00000000° N,...,Adult Unemployed,11:00,0.0,DİĞER,ESENLER,11:15,15.0,15.0,K1Y1,202767.0
3,6v8e5FWFp6A0rG05P5U9,58,75,Tek,6v8e5FWFp6A0rG05P5U9-K1,K1Y2E1,Eve gidiş,Kadın,Yürüme,0.00000000° N,...,Adult Unemployed,11:15,0.0,EV(SADECE İKAMET EDİLEN EV),ESENLER,11:25,10.0,10.0,K1Y2,202768.0
4,NAVrD2WQ3YeXoDthyEdd,59,76,Tek,NAVrD2WQ3YeXoDthyEdd-K1,K1Y1E1,Diğer,Erkek,Belediye Otobüsü,41.09473358° N,...,Adult Unemployed,11:00,0.0,AKTARMA YERİ,BEYKOZ,12:05,65.0,60.0,K1Y1,204082.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245249,yycdcequtEDveWUJFuKs,2232413,"2,193,816",NaN,yycdcequtEDveWUJFuKs-nan,D1Y2E1,Eve gidiş,NaN,Yürüme,0.00000000° N,...,NaN,21:00,0.0,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,21:55,55.0,55.0,NaN,NaN
245250,DkuNiQYoJeBPg5AYCj6H,2232537,"2,193,965",NaN,DkuNiQYoJeBPg5AYCj6H-nan,D3Y1E1,İşe gidiş,NaN,Yürüme,41.03251254° N,...,NaN,08:00,0.0,İŞ YERİ,GÜNGÖREN,08:15,15.0,15.0,NaN,NaN
245251,Qj247VNWaMK7v1mFC14Y,2232626,"2,194,067",NaN,Qj247VNWaMK7v1mFC14Y-nan,D4Y1E1,Diğer,NaN,Yürüme,41.03269375° N,...,NaN,12:00,0.0,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,12:30,30.0,30.0,NaN,NaN
245252,KwopuIgNhOqaYGT4F80V,2232685,"2,194,162",NaN,KwopuIgNhOqaYGT4F80V-nan,D1Y1E1,Sağlık Kurumuna gidiş,NaN,Özel Araç,41.03234608° N,...,NaN,15:00,0.0,DİĞER,BAHÇELİEVLER,16:45,105.0,95.0,NaN,NaN


In [18]:
test = df_selected[df_selected["p_id"] == "86"]
test

,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,mode,orig_lat,...,Employment Group,start_time,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time,k_y_part,same_p_and_k_y
6,dV2PuzpWEz5mlnPX077J,61,86,Tek,dV2PuzpWEz5mlnPX077J-K3,K3Y1E1,İşe gidiş,Kadın,Belediye Otobüsü,0.00000000° N,...,Full Time,07:30,2.0,AKTARMA YERİ,BEŞİKTAŞ,09:00,90.0,68.0,K3Y1,217175.0
7,dV2PuzpWEz5mlnPX077J,62,86,Tek,dV2PuzpWEz5mlnPX077J-K3,K3Y1E2,İşe gidiş,Kadın,Belediye Otobüsü,41.08704190° N,...,Full Time,18:00,10.0,İŞ YERİ,BEŞİKTAŞ,20:00,120.0,90.0,K3Y1,217175.0


### 🧹 Seyahat Süresi Verilerinin Temizlenmesi

Veri setindeki `travel_time` ve `vehicle_time` kolonları üzerinden aşağıdaki temizlik işlemleri uygulanmıştır:

1. **Eksik `travel_time` Verilerinin Silinmesi**  
   - Başlangıç (`start_time`) veya bitiş (`end_time`) verisi eksik olduğunda `travel_time` hesaplanamaz.
   - Bu nedenle `travel_time` değeri **NaN** olan satırlar veri setinden çıkarılmıştır.

2. **Negatif `vehicle_time` Değerlerinin Temizlenmesi**  
   - `vehicle_time`, `travel_time`'dan erişim (`access_time`), bekleme (`waiting_time`) ve varış geçişi (`egress_time`) sürelerinin çıkarılmasıyla elde edilir.
   - Teorik olarak bu değer negatif olamaz. Bu nedenle **0'dan küçük** `vehicle_time` satırları da silinmiştir.

3. **Eksik Değerlerin Kontrolü**  
   - Temizlik sonrası hâlâ `access_time`, `egress_time`, `travel_time`, `vehicle_time` kolonlarında eksik veri olup olmadığı kontrol edilmiştir.

Bu adım sayesinde analizlerde hatalı veya eksik sürelerden kaynaklı sapmaların önüne geçilmiştir.


In [19]:
df_selected = df_selected.dropna(subset=["same_p_and_k_y"])

# Önce travel_time kolonunda eksik veri olan same_p_and_k_y değerlerini tespit et
groups_with_missing_tt = df_selected[df_selected["travel_time"].isna()]["same_p_and_k_y"].unique()

# Bu değerlere sahip tüm satırları kaldır
df_selected = df_selected[~df_selected["same_p_and_k_y"].isin(groups_with_missing_tt)]


# 1. vehicle_time < 0 olan satırlardaki same_p_and_k_y değerlerini al
groups_with_negative_vt = df_selected[df_selected["vehicle_time"] < 0]["same_p_and_k_y"].unique()

# 2. Bu gruplara ait tüm satırları kaldır
df_selected = df_selected[~df_selected["same_p_and_k_y"].isin(groups_with_negative_vt)]


# Her sütunda kaç tane NaN (boş) değer olduğunu kontrol et
print("Access Time NaN sayısı:", df_selected["access_time"].isna().sum())
print("Egress Time NaN sayısı:", df_selected["egress_time"].isna().sum())
print("Travel Time NaN sayısı:", df_selected["travel_time"].isna().sum())
print("Vehicle Time NaN sayısı:", df_selected["vehicle_time"].isna().sum())


Access Time NaN sayısı: 0
Egress Time NaN sayısı: 0
Travel Time NaN sayısı: 0
Vehicle Time NaN sayısı: 0


### 🚦 Travel Time'a Göre Mode Seçimi

Bu bölümde, yolculuk verilerinde `travel_time` hesaplamaları yapılarak, ulaşım modlarının değerlendirilmesi ve seçiminde kullanılacak temel veriler hazırlanmıştı



Bu adımda, aynı `same_p_and_k_y` etiketine sahip (yani aynı kişi ve yolculuk kümesine ait) satırlar gruplanarak, her grubun içerisindeki mod türleri ve seyahat süreleri ayrı ayrı sütunlara aktarılmıştır:

- Örneğin: `mode_1`, `Travel_Time_1`, `mode_2`, `Travel_Time_2`, ...
- Bu yapı, bir yolculuk içinde ardışık olarak kullanılan ulaşım modlarını karşılaştırmalı analiz etmeyi kolaylaştırır.
- Veriler `result_df` adlı yeni bir DataFrame'de saklanmıştır.

Her satır artık bir yolculuk serisini temsil eder ve her mod tipi ile süresi yatay eksende gösterilir.
ını sağlar.


In [20]:
len(df_selected)

240800

In [21]:
df_selected['mode_order'] = df_selected.groupby('same_p_and_k_y').cumcount() + 1
# 3. Pivot işlemi ile her mod ve süresini ayrı kolona alalım
modes_pivot = df_selected.pivot(index='same_p_and_k_y', columns='mode_order', values='mode_category')
times_pivot = df_selected.pivot(index='same_p_and_k_y', columns='mode_order', values='travel_time')

# 4. Kolon adlarını düzenleyelim
modes_pivot.columns = [f'mode_{i}' for i in modes_pivot.columns]
times_pivot.columns = [f'Travel_Time_{i}' for i in times_pivot.columns]

# 5. İkisini birleştirelim
result_df = pd.concat([modes_pivot, times_pivot], axis=1).reset_index()
result_df.head(1)

,same_p_and_k_y,mode_1,mode_2,mode_3,mode_4,Travel_Time_1,Travel_Time_2,Travel_Time_3,Travel_Time_4
0,0.0,Walking,NaN,NaN,NaN,270.0,NaN,NaN,NaN


## 🧠 Ulaşım Modu Seçimi: `mode_choice` Hesaplama

Her `same_p_and_k_y` grubuna ait ardışık ulaşım modları (`mode_1`, `mode_2`, ...) ve bu modların seyahat süreleri (`Travel_Time_1`, `Travel_Time_2`, ...) incelenerek en baskın ulaşım modu belirlenmiştir. Bu karar, `mode_choice` kolonu olarak `result_df` veri setine eklenmiştir.

### 🔍 Karar Kuralları:

1. **Tüm modlar aynıysa**: Direkt olarak o mod seçilir.  
2. **Belirli kombinasyonlar için öncelikli tercihler**:
   - `Walking + Service` → **Service**
   - `Walking + Public Transport` → **Public Transport**
   - `Walking + Private Vehicle` → **Private Vehicle**
   - `Walking + Micromobility` → **Micromobility**
   - `Service + Private Vehicle + Public Transport` → **Public Transport**
   - `Service + Private Vehicle` → **Private Vehicle**
   - `Private Vehicle + Public Transport` → **Public Transport**
   - `Service + Public Transport` → Hangisinin toplam `travel_time` değeri fazlhip mod seçilir.

Bu algoritma, karma yolculuk zincirleri içinde en etkili veya baskın ulaşım modunu belirlemek amacıyla geliştirilmiştir.


In [22]:
import pandas as pd
import numpy as np

def mode_decision(row):
    """Her satır için mode_choice değerini döndürür."""
    # 1) Satırdaki modları ve süreleri topla
    travel_totals = {}          # {'Public Transport': 85, 'Service': 42, ...}
    modes_list    = []          # tekrar edenler dâhil ['Walking', 'Service', ...]

    for i in range(1, 5):       # mode_1…mode_4
        mode = row.get(f"mode_{i}")
        t    = row.get(f"Travel_Time_{i}")
        if pd.notna(mode):
            modes_list.append(mode)
            travel_totals[mode] = travel_totals.get(mode, 0) + (t if pd.notna(t) else 0)

    unique_modes = set(modes_list)

    # 2) Tüm modlar aynıysa doğrudan o modu seç
    if len(unique_modes) == 1:
        return next(iter(unique_modes))

    # 3) Tanımlı kombinasyon kuralları
    if unique_modes == {"Walking", "Service"}:
        return "Service"
    if unique_modes == {"Walking", "Public Transport"}:
        return "Public Transport"
    if unique_modes == {"Walking", "Private Vehicle"}:
        return "Private Vehicle"
    if unique_modes == {"Walking", "Micromobility"}:
        return "Micromobility"

    if {"Service", "Private Vehicle", "Public Transport"}.issubset(unique_modes):
        return "Public Transport"
    if unique_modes == {"Service", "Private Vehicle"}:
        return "Private Vehicle"
    if unique_modes == {"Private Vehicle", "Public Transport"}:
        return "Public Transport"
    if unique_modes == {"Service", "Public Transport"}:
        # Hangisinin toplam travel time’ı fazlaysa onu seç
        return (
            "Public Transport"
            if travel_totals.get("Public Transport", 0) >= travel_totals.get("Service", 0)
            else "Service"
        )

    # 4) Buraya düştüyse: en uzun toplam süreye sahip mod
    return max(travel_totals, key=travel_totals.get)

# ──────────────────────────────────────────────────────
# Yeni kolonun eklenmesi
result_df["mode_choice"] = result_df.apply(mode_decision, axis=1)
result_df.head(1)

,same_p_and_k_y,mode_1,mode_2,mode_3,mode_4,Travel_Time_1,Travel_Time_2,Travel_Time_3,Travel_Time_4,mode_choice
0,0.0,Walking,NaN,NaN,NaN,270.0,NaN,NaN,NaN,Walking


## 🔗 Mode Choice Bilgisinin Ana Veriye Eklenmesi

Önceki adımda `result_df` veri çerçevesi üzerinde hesaplanan `mode_choice` bilgisi, her `same_p_and_k_y` grubu için baskın ulaşım modunu temsil eder.  
Bu adımda, bu bilgi tekrar `df_selected` veri setine aktarılmıştır:

- `same_p_and_k_y` kolonu üzerinden `result_df` ve `df_selected` birleştirilmiştir.
- Böylece her yolculuk satırına karşılık gelen `mode_choice` değeri eklenmiştir.

Bu işlem sayesinde, her yolculuk alt satırının analizinde baskın ulaşım türüne göre gruplayarak işlem yapılabilir hâle gelinmiştir.


In [23]:
mode_choice_map = result_df[["same_p_and_k_y", "mode_choice"]]

# df_selected'e aynı same_p_and_k_y'ye göre mode_choice ekle
df_selected = df_selected.merge(mode_choice_map, on="same_p_and_k_y", how="left")

In [24]:
df_selected.head(1)

,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,mode,orig_lat,...,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time,k_y_part,same_p_and_k_y,mode_order,mode_choice
0,6kaqzPZwvLgUnuUalhTx,55,72,Tek,6kaqzPZwvLgUnuUalhTx-K2,K2Y1E1,Alışveriş,Erkek,Yürüme,41.09473358° N,...,0.0,DİĞER,BEYKOZ,09:30,10.0,10.0,K2Y1,198914.0,1,Walking


## 📊 Mode Choice Dağılımının Hesaplanması (Test-TripLeg)

Bu adımda, hesaplanan `mode_choice` kolonunun genel dağılımı analiz edilmiştir:

1. **Temizlik**  
   - `mode_choice` değeri `"Other"` olan satırlar analiz dışı bırakılmıştır. Bu değer, belirli kurallara uymayan veya eksik verilerden oluşan grupları temsil eder.

2. **Tekilleştirme**  
   - Aynı `same_p_and_k_y` değerine sahip tekrar eden satırlar kaldırılarak, her yolculuk grubu yalnızca 1 kez temsil edilmiştir.

3. **Yüzdelik Dağılım**  
   - Kalan `mode_choice` değerlerinin oranları yüzdelik olarak hesaplanmış ve 2 ondalık basamağa yuvarlanmıştır.

Bu dağılım, hangi ulaşım modlarının daha baskın kullanıldığını anlamak açısından önemlidir.


In [25]:
df_selected = df_selected[df_selected['mode_choice'] != "Other"]
# Her same_p_and_k_y için yalnızca 1 satır al
unique_modes = df_selected[['same_p_and_k_y', 'mode_choice']].drop_duplicates()
distribution_percent = unique_modes['mode_choice'].value_counts(normalize=True, dropna=False) * 100
print(distribution_percent.round(2))


mode_choice
Walking             53.28
Public Transport    19.58
Private Vehicle     18.64
Service              8.16
Micromobility        0.34
Name: proportion, dtype: float64


## ⏳ Yolculuk Sürelerinin Gruplandırılması ve Özetlenmesi

Bu bölümde yolculuk verileri (`df_selected`) üzerinde iki ana işlem yapılmıştır:

1. **Gruplama ve Toplama (`agg_df_4`)**  
   - Her `same_p_and_k_y` etiketi için aşağıdaki değerler **toplanmıştır**:
     - `travel_time`, `vehicle_time`, `access_time`, `egress_time`, `waiting_time`
   - Ayrıca tanımlayıcı bilgilerden ilki (`district`, `p_id`, `id`) korunmuştur.  
   - Sonuç, gruplandırılmış özet veri çerçevesi **`agg_df_4`** olarak saklanmıştır.

2. **Travel Time Aralık Sınıflandırması (`travel_time_group`)**  
   - **0 – 360+ dakika** aralığında özel kademeli sınıflar tanımlanmıştır  
     (daha sık <30 dk aralıkları, ardından 15 dk’lık kademeler).  
   - `pd.cut()` ile her grubun toplam `travel_time` süresi, **`travel_time_group`** kategorisine atanmıştır.
   - Örnek etiketler:  
     - `0`, `0-1`, `1-5`, `5-10`, `10-15`, `15-20`, `20-25`, `25-30`,  
       `30-45`, `45-60`, … , `345-360`, `360+`.

Bu sınıflandırma, farklı yolculuk süreleri için sayısal dağılım analizi ve görselleştirmeyi kolaylaştırır.


In [26]:
import pandas as pd

# Örnek olarak df_selected yüklü varsayılıyor
# df_selected = pd.read_csv("your_file.csv") gibi yüklenebilir

# Gruplama ve toplama işlemi
agg_df_4 = df_selected.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "district": "first",
    "p_id": "first",
    "id": "first"
    
})

# Güncellenmiş travel_time grupları: 15-30 yerine 15-20, 20-25, 25-30
bins = (
    [0, 1, 5, 10, 15, 20, 25, 30] + 
    list(range(45, 301, 15)) + 
    [315, 330, 345, 360, float('inf')]
)

# Etiketler (labels)
# Bins (aralık uçları)
bins = (
    [0, 0.000001, 1, 5, 10, 15, 20, 25, 30] + 
    list(range(45, 301, 15)) + 
    [315, 330, 345, 360, float('inf')]
)

# Labels
labels = (
    ['0', '0-1', '1-5', '5-10', '10-15', '15-20', '20-25', '25-30'] + 
    [f"{i}-{i+15}" for i in range(30, 300, 15)] + 
    ['300-315', '315-330', '330-345', '345-360', '360+']
)

# travel_time değerlerini sınıflandırma
agg_df_4["travel_time_group"] = pd.cut(agg_df_4["travel_time"], bins=bins, labels=labels, right=False)
agg_df_4.head(1)


,same_p_and_k_y,travel_time,vehicle_time,access_time,egress_time,waiting_time,district,p_id,id,travel_time_group
0,0.0,270.0,270.0,0.0,0.0,0.0,KADIKÖY,"1,000,140",1000115,270-285


## ✅ Geçerli Travel Time Gruplarının Filtrelenmesi

Bu adımda, sadece belirli aralıklardaki yolculuk sürelerine sahip gruplar analiz için seçilmiştir.

### 🎯 Seçilen Süre Grupları:
`1-5`, `5-10`, `10-15`, `15-20`, `20-25`, `25-30`,  
`30-45`, `45-60`, `60-75`, `75-90`, `90-105`, `105-120`,  
`120-135`, `135-150`, `150-165`, `165-180`, `180-195`,  
`195-210`, `210-225`, `225-240`

Bu aralıklar, hem kısa hem de orta/uzun mesafeli yolculukları kapsayan **dengeli bir dağılım** sağlar.  
Amaç, veri dışlayıcı uç değerlerden (örneğin: 0-1 dakika veya 360+ dakika) arındırılmış, analiz için güvenilir bir örneklem elde etmektir.

Sonuç, `agg_df_4_filtered` adlı yeni veri çerçevesinde saklanmıştır.


In [27]:
gecerli_gruplar = [
    "1-5", "5-10", "10-15", "15-20", "20-25", "25-30", "30-45",
    "45-60", "60-75", "75-90", "90-105", "105-120", "120-135",
    "135-150", "150-165", "165-180", "180-195", "195-210", 
    "210-225", "225-240"
]

# Sadece belirtilen travel_time_group değerlerine sahip satırları al
agg_df_4_filtered = agg_df_4[agg_df_4["travel_time_group"].isin(gecerli_gruplar)]

# 1. agg_df_4_filtered'deki geçerli same_p_and_k_y değerlerini al
gecerli_idler = agg_df_4_filtered["same_p_and_k_y"].unique()
# 2. df_selected'da sadece bu id'lere sahip satırları tut
df_selected_v2 = df_selected[df_selected["same_p_and_k_y"].isin(gecerli_idler)]
df_selected_v2.head(1)

,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,mode,orig_lat,...,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time,k_y_part,same_p_and_k_y,mode_order,mode_choice
0,6kaqzPZwvLgUnuUalhTx,55,72,Tek,6kaqzPZwvLgUnuUalhTx-K2,K2Y1E1,Alışveriş,Erkek,Yürüme,41.09473358° N,...,0.0,DİĞER,BEYKOZ,09:30,10.0,10.0,K2Y1,198914.0,1,Walking


<h1 style="color:#2E86C1; font-size:32px;">📊 ANALİZ AŞAMASI</h1>

---

🎯 **Bu bölüm, çalışmanın en kritik kısmıdır.**  
Ön işleme ve veri temizliği başarıyla tamamlandı. Artık temiz ve güvenilir veri kümesi ile içgörü üretmeye hazılaştırması

📌 Her analiz bloğu **adım adım açıklanacak**, gerekli görülen noktalarda **grafikler ve tablolarla** destekkte keşfedelim.


<span style="font-size:22pt; color:red;"><b>Figure 85. Travel Time based on Mode (Simplified) (İstanbul SUMP Stage II HTS)</b></span>

### ⏱️ Süre Bileşenlerinin Ayrıştırılması ve Dengeleme

Bu adım, her **yolculuk grubunun** (`same_p_and_k_y`, `mode_category`) toplam seyahat süresini bileşenlerine ayırır ve tutarsızlıkları düzeltir.

### Adımlar
1. **`id_int` Temizleme**  
   Virgüllü `id` değerleri temizlenip tam sayıya (`int`) dönüştürüldü.  
   Bu, kronolojik sıralama için gereklidir.

2. **Özel Toplama Fonksiyonu (`custom_agg`)**  
   - **Toplamlar** `travel_time`, `vehicle_time`, `waiting_time`  
   - **Access Time** En küçük `id_int` satırından alınır.  
   - **Egress Time** En büyük `id_int` satırından alınır.  
   - **Transfer Time** Grup içindeki **ilk (n − 1)** satırın `egress_time` toplamı.

3. **Gruplama**  
   `same_p_and_k_y` ✕ `mode_category` düzeyinde `custom_agg` uygulanarak **`PT`** tablosu oluşturuldu.

4. **Fark Kontrolü ve Dengeleme**  
   - `difference` = `travel_time` − (diğer bileşenlerin toplamı)  
   - Eğer `difference` > 0 ise bu tutarsızlık **`vehicle_time`**’a eklenerek denge sağlandı.  
   - Son olarak `difference` sütunu kaldırıldı.

> Bu işlem, süre bileşenlerinin mantıksal tutarlılığını garanti eder ve sonraki analiz adımlarında güvenilir sonuçlar elde edilmesini sağlar.


In [28]:
import pandas as pd

# id kolonunu temizle: string'e çevir, virgülü kaldır, int'e çevir
df_selected_v2['id_int'] = (
    df_selected_v2['id']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(int)
)

# Özel gruplama fonksiyonu tanımı
def custom_agg(group):
    travel_time_sum = group['travel_time'].sum()
    vehicle_time_sum = group['vehicle_time'].sum()
    waiting_time_sum = group['waiting_time'].sum()
    
    # access_time → en küçük id'ye sahip satır
    access_row = group.loc[group['id_int'].idxmin()]
    access_time_value = access_row['access_time']
    
    # egress_time → en büyük id'ye sahip satır
    egress_row = group.loc[group['id_int'].idxmax()]
    egress_time_value = egress_row['egress_time']
    
    # transfer_time → ilk (n-1) egress_time toplamı
    group_sorted = group.sort_values(by='id_int')
    n_rows = len(group_sorted)
    
    if n_rows == 1:
        transfer_time_value = 0
    else:
        transfer_time_value = group_sorted.iloc[:-1]['egress_time'].sum()
    
    return pd.Series({
        'travel_time': travel_time_sum,
        'vehicle_time': vehicle_time_sum,
        'waiting_time': waiting_time_sum,
        'access_time': access_time_value,
        'egress_time': egress_time_value,
        'transfer_time': transfer_time_value
    })

# Gruplama: same_p_and_k_y ve mode_category'ye göre
PT = df_selected_v2.groupby(['same_p_and_k_y', 'mode_category']).apply(custom_agg).reset_index()

# Farkı hesapla: travel_time ile bileşenlerin toplamı arasında fark
PT['difference'] = (
    PT['travel_time']
    - PT['vehicle_time']
    - PT['waiting_time']
    - PT['access_time']
    - PT['egress_time']
    - PT['transfer_time']
)

# Eğer fark > 0 ise vehicle_time'a ekle
PT.loc[PT['difference'] > 0, 'vehicle_time'] += PT['difference']

# difference kolonunu kaldır
PT = PT.drop(columns=['difference'])



C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\1447449726.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2['id_int'] = (


### 🚍 Toplu Taşıma Yolculukları için Ortalama Süre Bileşenleri

Bu analizde, yalnızca **Public Transport** (toplu taşıma) içeren yolculuk grupları filtrelenerek, aşağıdaki süre bileşenlerinin **ortalama değerleri** hesaplanmıştır:

- **travel_time**: Toplam seyahat süresi  
- **vehicle_time**: Araç içinde geçirilen süre  
- **access_time**: Toplu taşımaya erişim süresi  
- **egress_time**: Toplu taşımadan indikten sonraki ulaşım süresi  
- **waiting_time**: Durağa/istasyona ulaştıktan sonra araç bekleme süresi  
- **transfer_time**: Aktarma yapılan duraklarda geçirilen süre

Sonuçlar `pt_means` tablosunda yer almakta olup, tüm değerler **dakika cinsindendir** ve **1 ondalık basamak** ile yuvarlanmıştır.

Bu özet, toplu taşıma kullanıcılarının zamanlarının ne kadarını doğrudan araçta, ne kadarını erişim-bekleme-aktarma gibi dış faktörlerle geçirdiğini anlamaya yardımcı olur.


In [29]:
pt_means = (PT[PT['mode_category'] == 'Public Transport']
            .groupby("mode_category")[[
                "travel_time",
                "vehicle_time",
                "access_time",
                "egress_time",
                "waiting_time",
                "transfer_time"
            ]]
            .mean()
            .round(1)
            .reset_index())
pt_means

,mode_category,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
0,Public Transport,52.9,33.0,6.2,6.8,6.5,0.4


In [30]:
# transfer_time > 0 olan satırları filtrele
pt_with_transfer = PT[
    (PT["mode_category"] == "Public Transport") &
    (PT["transfer_time"] > 0)
]

# Ortalama değerleri hesapla
pt_transfer_mean = pt_with_transfer[[
    "transfer_time"
]].mean().round(1)
pt_transfer_mean



transfer_time    4.4
dtype: float64

### 🚶‍♀️ Alternatif Ulaşım Modları için Ortalama Süre Bileşenleri

Toplu taşıma dışındaki ulaşım modları için (Service, Private Vehicle, Walking, Micromobility) **ortalama süre bileşenleri** hesaplanmıştır.

### 🔍 İncelenen Modlar:
- **Service** (Servis aracı)
- **Private Vehicle** (Özel araç, taksi, motosiklet vb.)
- **Walking** (Yaya)
- **Micromobility** (Bisiklet, skuter, paylaşımlı skuter)

### 📊 Hesaplanan Ortalama Değerler:
- **travel_time**: Toplam seyahat süresi  
- **vehicle_time**: Araçta/yolda geçirilen aktif süre  
- **access_time**: Başlangıç noktasına ulaşım süresi  
- **egress_time**: Varış noktasından hedefe erişim süresi  
- **waiting_time**: Başlamadan önce beklenen süre

Sonuçlar, her mod için bu bileşenlerin **ortalama değerleri** ile `means` tablosunda sunulmuştur (dakika cinsinden, 1 ondalık basamakla).


In [31]:
# İstediğimiz mode_category'leri seçelim
selected_modes = ['Service', 'Private Vehicle', 'Walking','Micromobility']

# Filtrele ve ortalamaları hesapla
means = (df_selected_v2[df_selected_v2['mode_category'].isin(selected_modes)]
            .groupby("mode_category")[[
                "travel_time",
                "vehicle_time",
                "access_time",
                "egress_time",
                "waiting_time",
            ]]
            .mean()
            .round(1)
            .reset_index())
means

,mode_category,travel_time,vehicle_time,access_time,egress_time,waiting_time
0,Micromobility,27.5,27.4,0.0,0.1,0.0
1,Private Vehicle,37.5,32.8,2.0,2.6,0.1
2,Service,45.3,38.0,2.4,2.9,2.0
3,Walking,17.3,17.3,0.0,0.0,0.0


### 🚶‍♀️ Alternatif Ulaşım Modları için Ortalama Süre Bileşenleri SADECE İSTANBUL

In [32]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# df_selected_v2'yi değiştirmeden, filtrelenmiş yeni bir DataFrame oluştur
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# id kolonunu temizle: string'e çevir, virgülü kaldır, int'e çevir
df_filtered['id_int'] = (
    df_filtered['id']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(int)
)

# Özel gruplama fonksiyonu tanımı
def custom_agg(group):
    travel_time_sum = group['travel_time'].sum()
    vehicle_time_sum = group['vehicle_time'].sum()
    waiting_time_sum = group['waiting_time'].sum()
    
    access_row = group.loc[group['id_int'].idxmin()]
    access_time_value = access_row['access_time']
    
    egress_row = group.loc[group['id_int'].idxmax()]
    egress_time_value = egress_row['egress_time']
    
    group_sorted = group.sort_values(by='id_int')
    n_rows = len(group_sorted)
    
    if n_rows == 1:
        transfer_time_value = 0
    else:
        transfer_time_value = group_sorted.iloc[:-1]['egress_time'].sum()
    
    return pd.Series({
        'travel_time': travel_time_sum,
        'vehicle_time': vehicle_time_sum,
        'waiting_time': waiting_time_sum,
        'access_time': access_time_value,
        'egress_time': egress_time_value,
        'transfer_time': transfer_time_value
    })

# Gruplama: same_p_and_k_y ve mode_category'ye göre
PT = df_filtered.groupby(['same_p_and_k_y', 'mode_category']).apply(custom_agg).reset_index()

# travel_time ile bileşenlerin toplamı arasındaki farkı hesapla
PT['difference'] = (
    PT['travel_time']
    - PT['vehicle_time']
    - PT['waiting_time']
    - PT['access_time']
    - PT['egress_time']
    - PT['transfer_time']
)

# Eğer fark > 0 ise vehicle_time'a ekle
PT.loc[PT['difference'] > 0, 'vehicle_time'] += PT['difference']

# difference kolonunu kaldır
PT = PT.drop(columns=['difference'])


In [33]:
pt_means = (PT[PT['mode_category'] == 'Public Transport']
            .groupby("mode_category")[[
                "travel_time",
                "vehicle_time",
                "access_time",
                "egress_time",
                "waiting_time",
                "transfer_time"
            ]]
            .mean()
            .round(2)
            .reset_index())
pt_means

,mode_category,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
0,Public Transport,53.2,33.21,6.26,6.86,6.49,0.38


In [34]:
# transfer_time > 0 olan satırları filtrele
pt_with_transfer = PT[
    (PT["mode_category"] == "Public Transport") &
    (PT["transfer_time"] > 0)
]

# Ortalama değerleri hesapla
pt_transfer_mean = pt_with_transfer[[
    "transfer_time"
]].mean().round(1)
pt_transfer_mean



transfer_time    4.4
dtype: float64

In [35]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçelerine göre filtreleme
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)]

# İstediğimiz mode_category'leri seçelim
selected_modes = ['Service', 'Private Vehicle', 'Walking', 'Micromobility']

# Filtrele ve ortalamaları hesapla
means = (
    df_filtered[df_filtered['mode_category'].isin(selected_modes)]
    .groupby("mode_category")[[
        "travel_time",
        "vehicle_time",
        "access_time",
        "egress_time",
        "waiting_time",
    ]]
    .mean()
    .round(2)
    .reset_index()
)

means


,mode_category,travel_time,vehicle_time,access_time,egress_time,waiting_time
0,Micromobility,27.66,27.56,0.02,0.08,0.00
1,Private Vehicle,37.78,33.04,2.00,2.62,0.13
2,Service,46.45,39.17,2.34,2.86,2.08
3,Walking,17.37,17.37,0.00,0.00,0.00


<span style="font-size:22pt; color:red;"><b>Mode Kırımları İçin  Tablo</b></span>

### 


In [36]:
agg_df_5 = df_selected_v2.groupby(["same_p_and_k_y", "mode"], as_index=False).agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "district": "first",
    "p_id": "first",
    "id": "first"
})
agg_df_5.head(1)

,same_p_and_k_y,mode,travel_time,vehicle_time,access_time,egress_time,waiting_time,district,p_id,id
0,1.0,Yürüme,15.0,15.0,0.0,0.0,0.0,GÜNGÖREN,"1,000,168",1000143


<span style="font-size:22pt; color:red;"><b>Figure 86. Distribution of In Travel Times by Transportation Modes (İstanbul SUMP Stage II HTS)</b></span>

### 

In [37]:
# Travel time için gruplar belirleniyor
bins = [0, 10, 20, 30, 40, 50, 60, float("inf")]
labels = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60+"]

# travel_time değerlerini sınıflandır
df_selected_v2["travel_time_group"] = pd.cut(df_selected_v2["travel_time"], bins=bins, labels=labels, right=False)

# mode ve travel_time_group'a göre sayım yap
grouped_counts = df_selected_v2.groupby(["mode", "travel_time_group"]).size().unstack(fill_value=0)

# Sonucu göster
from IPython.display import display
display(grouped_counts)

C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\291722006.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2["travel_time_group"] = pd.cut(df_selected_v2["travel_time"], bins=bins, labels=labels, right=False)


travel_time_group,0-10,10-20,20-30,30-40,40-50,50-60,60+
mode,,,,,,,
Belediye Otobüsü,41,658,2415,5340,4460,2149,10294
Bisiklet,34,304,80,104,22,7,99
Deniz Motoru,0,0,3,5,2,1,12
Deniz Otobüsü (IDO),0,2,3,4,10,3,11
Deniz Taksi,0,0,0,0,0,0,1
Diğer,7,54,41,102,33,4,66
Dolmuş,3,63,246,506,341,99,360
Füniküler,0,0,2,2,1,0,1
Halkalı-Bahçeşehir Raylı Sistem Hattı,0,1,0,2,1,2,3


In [38]:
grouped_transposed = grouped_counts.T

# 2. Indexleri sıfırla (mode sütunu yeniden gelir)
grouped_transposed_reset = grouped_transposed.reset_index()
grouped_transposed_reset.to_excel("figure86.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 87. Access Time Distribution by Transportation Modes (İstanbul SUMP Stage II HTS)</b></span>

###  

In [39]:
# Travel time için gruplar belirleniyor
bins = [0, 10, 20, 30, 40, 50, 60, float("inf")]
labels = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60+"]

# travel_time değerlerini sınıflandır
df_selected_v2["acces_group"] = pd.cut(df_selected_v2["access_time"], bins=bins, labels=labels, right=False)

# mode ve travel_time_group'a göre sayım yap
grouped_counts_2 = df_selected_v2.groupby(["mode", "acces_group"]).size().unstack(fill_value=0)

# Sonucu göster
from IPython.display import display
display(grouped_counts_2)


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\2309693449.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2["acces_group"] = pd.cut(df_selected_v2["access_time"], bins=bins, labels=labels, right=False)


acces_group,0-10,10-20,20-30,30-40,40-50,50-60,60+
mode,,,,,,,
Belediye Otobüsü,19888,5182,250,29,5,1,2
Bisiklet,650,0,0,0,0,0,0
Deniz Motoru,12,6,3,2,0,0,0
Deniz Otobüsü (IDO),20,11,2,0,0,0,0
Deniz Taksi,1,0,0,0,0,0,0
Diğer,297,10,0,0,0,0,0
Dolmuş,1279,323,13,3,0,0,0
Füniküler,6,0,0,0,0,0,0
Halkalı-Bahçeşehir Raylı Sistem Hattı,6,2,1,0,0,0,0


In [40]:
grouped_transposed_2= grouped_counts_2.T

# 2. Indexleri sıfırla (mode sütunu yeniden gelir)
grouped_transposed_reset_2= grouped_transposed_2.reset_index()
grouped_transposed_reset_2.to_excel("acces_time.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 88. Egress Time Distribution by Transportation Modes (İstanbul SUMP Stage II HTS)</b></span>


### 

In [41]:
# Travel time için gruplar belirleniyor
bins = [0, 10, 20, 30, 40, 50, 60, float("inf")]
labels = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60+"]

# travel_time değerlerini sınıflandır
df_selected_v2["egress_group"] = pd.cut(df_selected_v2["egress_time"], bins=bins, labels=labels, right=False)

# mode ve travel_time_group'a göre sayım yap
grouped_counts_3 = df_selected_v2.groupby(["mode", "egress_group"]).size().unstack(fill_value=0)

# Sonucu göster
from IPython.display import display
display(grouped_counts_3)


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\1225928935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2["egress_group"] = pd.cut(df_selected_v2["egress_time"], bins=bins, labels=labels, right=False)


egress_group,0-10,10-20,20-30,30-40,40-50,50-60,60+
mode,,,,,,,
Belediye Otobüsü,19274,5619,337,80,34,7,6
Bisiklet,650,0,0,0,0,0,0
Deniz Motoru,10,8,3,2,0,0,0
Deniz Otobüsü (IDO),14,17,1,1,0,0,0
Deniz Taksi,1,0,0,0,0,0,0
Diğer,275,14,6,8,2,0,2
Dolmuş,1263,328,21,5,1,0,0
Füniküler,5,1,0,0,0,0,0
Halkalı-Bahçeşehir Raylı Sistem Hattı,4,4,1,0,0,0,0


In [42]:
grouped_transposed_3= grouped_counts_3.T

# 2. Indexleri sıfırla (mode sütunu yeniden gelir)
grouped_transposed_reset_3= grouped_transposed_3.reset_index()
grouped_transposed_reset_3.head(1)

mode,egress_group,Belediye Otobüsü,Bisiklet,Deniz Motoru,Deniz Otobüsü (IDO),Deniz Taksi,Diğer,Dolmuş,Füniküler,Halkalı-Bahçeşehir Raylı Sistem Hattı,...,Paylaşımlı Skuter,Servis,Skuter,Taksi,Teleferik,Tramvay,Tünel,Yürüme,Özel Araç,Şehir Hatları Vapuru
0,0-10,19274,650,10,14,1,275,1263,5,4,...,23,18160,120,1591,2,1431,1,124677,36740,66


<span style="font-size:22pt; color:red;"><b>Figure 89. In Vehicle Time based on Person Type for Public Transport (İstanbul SUMP Stage II HTS)</b></span>

### 🚍 Public Transport Kullanıcıları: Employment Group Bazında Süre Analizi

Bu bölümde sadece **Public Transport** (toplu taşıma) kullanan bireyler filtrelenerek, çalışma/öğrenim durumuna göre ortalama süre analizleri yapılmıştır.

### 📦 İşlem Adımları:

1. **Toplu Taşıma Kullananların Filtrelenmesi**  
   - `mode_category == "Public Transport"` olan satırlar seçilerek `df_public_transport` oluşturulmuştur.

2. **Gruplama ve Toplama (`agg_df_7`)**  
   - Her bireyin (`same_p_and_k_y`) ve ait olduğu **Employment Group** bazında:
     - `travel_time`, `vehicle_time`, `access_time`, `egress_time`, `waiting_time` toplamları hesaplanmıştır.

3. **Ortalama Süre Hesaplaması (`person_type_PT`)**  
   - Her çalışma/öğrenim grubu için yukarıdaki süre bileşenlerinin **ortalama değerleri** elde edilmiştir.

Bu analiz, farklı sosyal grupların **toplu taşımayı ne şekilde ve ne kadar süre ile kullandığını** kıyaslamaya olanak tanır.


In [43]:
df_public_transport = df_selected_v2[df_selected_v2["mode_category"] == "Public Transport"]
agg_df_7 = df_public_transport.groupby(["same_p_and_k_y", "Employment Group"], as_index=False).agg({
    "travel_time": "sum",
    "vehicle_time" : "sum",
    "district": "first",
    "p_id": "first",
    "id": "first"
})

# mode'a göre grupla ve ortalamaları al
person_type_PT = agg_df_7.groupby("Employment Group")[[
    "travel_time",
   "vehicle_time"
]].mean().reset_index()

# Sonucu yazdır
person_type_PT

,Employment Group,travel_time,vehicle_time
0,65+ Retired,48.682016,29.804169
1,Adult Student,60.181186,38.411383
2,Adult Unemployed,49.475711,30.889018
3,Child,52.887417,34.490066
4,Child Student,44.640255,26.483639
5,Full Time,53.989969,33.641674
6,Other,47.509036,26.030120
7,Part Time,57.463047,35.078431


<span style="font-size:22pt; color:red;"><b>Figure 140. Average Travel Times by Person Type for All Modes (İstanbul SUMP Stage II HTS)</b></span>
### 👤 Employment Group (Çalışma Durumu) Bazında Ortalama Süre Analizi

Bu analizde, her bireyin ait olduğu **Employment Group** (Çalışma/Öğrenim Durumu) kategorisine göre seyahat sürelerinin ortalamaları hesaplanmıştır.

### 📦 İşlem Adımları:

1. **Yolculuk Verisinin Gruplanması (`agg_df_6`)**  
   - `same_p_and_k_y` ve `Employment Group` kırılımında tüm süre bileşenleri toplanmıştır.
   - Ek olarak tanımlayıcı kolonlar (`district`, `p_id`, `id`) da ilk değer olarak alınmıştır.

2. **Ortalama Sürelerin Hesaplanması (`person_type`)**  
   - Her **Employment Group** için aşağıdaki sürelerin ortalamaları hesaplanmıştır:
     - `travel_time`
     - `vehicle_time`
     - `access_time`
     - `egress_time`
     - `waiting_time`

### 👥 İncelenen Gruplar Örneğin:
- Full Time
- Part Time
- Adult Student
- Adult Unemployed
- 65+ Retired
- Child Student
- Child
- Other

Bu sonuçlar, farklı toplumsal grupların şehir içi ulaşımda ne kadar süre harcadığını karşılaştırmalı olarak anlamamıza olanak sağlar.


In [44]:

agg_df_6 = df_selected_v2.groupby(["same_p_and_k_y", "Employment Group"], as_index=False).agg({
    "travel_time": "sum",
    "vehicle_time" : "sum"
})


# mode'a göre grupla ve ortalamaları al
person_type = agg_df_6.groupby("Employment Group")[[
    "travel_time",
    "vehicle_time"
]].mean().reset_index()

# Sonucu yazdır
person_type


,Employment Group,travel_time,vehicle_time
0,65+ Retired,21.645664,18.740028
1,Adult Student,48.674769,34.728989
2,Adult Unemployed,22.956900,19.917201
3,Child,24.956339,22.033585
4,Child Student,22.350435,19.557874
5,Full Time,38.675970,31.026300
6,Other,17.002134,15.189719
7,Part Time,39.869684,32.081155


 <span style="font-size:22pt; color:red;"><b>Table 24. Travel Times by Trip Purpose for All Trips and Vehicle Trips</b></span>

### 🏠➡️🏢 Evden İşe Yolculuklarda Ortalama Seyahat Süresi Analizi

Bu analizde sadece **evden işe gidiş** amacıyla yapılan yolculuklar filtrelenmiştir.

### 📦 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)`
- **Yolculuk Amacı**: `İşe gidiş`

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Ardından, bu değerlerin:
  - **Ortalaması** (`mean_tt`)
  - **Standart sapması** (`std_tt`)
  hesaplanmıştır.

Bu analiz, şehirde yaşayan bireylerin işe ulaşım sürelerinin genel düzeyini ve dağılımını anlamak için kritik bir içgörü sağlar.



In [45]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# Filtreleme
home_work = df_selected_v2[
    (df_selected_v2['orig_loc_type'] == 'EV(SADECE İKAMET EDİLEN EV)') &
    (df_selected_v2['purpose'] == 'İşe gidiş') &
    (df_selected_v2['district'].isin(istanbul_ilceleri))
]

# Gruplama ve istatistik
home_work_2 = home_work.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})

mean_tt = home_work_2['travel_time'].mean()
std_tt = home_work_2['travel_time'].std()

print("Ortalama travel_time:", mean_tt)
print("Standart sapma (std):", std_tt)


Ortalama travel_time: 37.37952385304332
Standart sapma (std): 24.27734281055003


### 🏠➡️🎓 Evden Okula Yolculuklarda Ortalama Seyahat Süresi Analizi

Bu analizde yalnızca **ikamet edilen evden** başlayan ve **okula gidiş amacı** taşıyan yolculuklar incelenmiştir.

### 🎯 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)`
- **Yolculuk Amacı**:
  - `Okula gidiş(ilk ve ortaöğretim)`
  - `Okula gidiş(lise)`
  - `Okula gidiş(üniversite ve üzeri)`

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_2`)
  - **Standart sapma** (`std_tt_2`)
  elde edilmiştir.

Bu analiz, öğrencilerin evden okula ulaşım süreleri hakkında genel bir fikir verir ve yaş gruplarına göre erişilebilirlik karşılaştırmaları için temel oluşturur.


In [46]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

home_school = df_selected_v2[
    (df_selected_v2['orig_loc_type'] == 'EV(SADECE İKAMET EDİLEN EV)') &
    (df_selected_v2['district'].isin(istanbul_ilceleri)) &
    (df_selected_v2['purpose'].isin([
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]))
]

home_school_2 = home_school.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})

mean_tt_2 = home_school_2['travel_time'].mean()
std_tt_2 = home_school_2['travel_time'].std()

print("Ortalama travel_time:", mean_tt_2)
print("Standart sapma (std):", std_tt_2)


Ortalama travel_time: 25.839155121508064
Standart sapma (std): 19.798296337123006


### 🏠➡️🧭 Evden Diğer Amaçlarla Yapılan Yolculuklarda Ortalama Seyahat Süresi Analizi

Bu analiz, evden başlayıp iş ya da okul dışındaki çeşitli amaçlarla yapılan yolculukları kapsamaktadır.

### 🎯 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)`
- **Yolculuk Amaçları** (örnekler):
  - `Akraba/Arkadaş Ziyareti`
  - `Alışveriş`
  - `Yeme/içme Gidiş`
  - `Sağlık Kurumuna gidiş`
  - `Spor, Eğlence vb. gidiş`
  - `Çocuk Alma/Bırakma`
  - `İş takibi/Ödemeler`
  - ve diğer serbest zaman/destek amaçlı yolculuklar

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_3`)
  - **Standart sapma** (`std_tt_3`) hesaplanmıştır.

Bu analiz, günlük hayatın farklı yönleriyle ilişkili yolculukların zaman maliyetini anlamak için yapılmıştır.


In [47]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]


home_other = df_selected_v2[
    (df_selected_v2['orig_loc_type'] == 'EV(SADECE İKAMET EDİLEN EV)') &
      (df_selected_v2['district'].isin(istanbul_ilceleri)) &
    (df_selected_v2['purpose'].isin([
    'Akraba/Arkadaş  Ziyareti',
    'Çocuk Alma/Bırakma',
    'Diğer',
    'Diğer Alma/Bırakma',
    'Eve gidiş',
    'Sağlık Kurumuna gidiş',
    'Alışveriş',
    'Diğer',
    'İş takibi/ Ödemeler',
    'Spor, Eğlence vb. gidiş',
    'Yeme/içme Gidiş'
    ]))
]


home_other_2 = home_other.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})
mean_tt_3 = home_other_2['travel_time'].mean()
std_tt_3 = home_other_2['travel_time'].std()
print("Ortalama travel_time:", mean_tt_3)
print("Standart sapma (std):", std_tt_3)

Ortalama travel_time: 22.485918461569344
Standart sapma (std): 22.150017179146154


### 🧳 Ev Dışı Başlangıç Noktalı Yolculuklarda Ortalama Seyahat Süresi Analizi

Bu analizde, **başlangıç noktası ev olmayan** yolculuklar incelenmiştir.

### 🎯 Filtreleme Kriteri:
- **Başlangıç Lokasyonu ≠** `EV(SADECE İKAMET EDİLEN EV)`

Bu tür yolculuklar şunları kapsayabilir:
- Aktarma noktalarından başlayan devam yolculukları
- İş, okul veya diğer mekânlardan başlayan dönüş veya yan yolculuklar
- Gün içinde yapılan zincirli seyahatlerin ara bölümleri

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_4`)
  - **Standart sapma** (`std_tt_4`) hesaplanmıştır.

Bu analiz, ev dışı kökenli yolculukların süresel karakterini ortaya koyarak zincirli ulaşım yapılarının anlaşılmasına katkı sağlar.


In [48]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# EV olmayan ve İstanbul ilçelerindeki kayıtları filtrele
other = df_selected_v2[
    (df_selected_v2['orig_loc_type'] != 'EV(SADECE İKAMET EDİLEN EV)') &
    (df_selected_v2['district'].isin(istanbul_ilceleri))
]

# İlçeye göre gruplama ve toplam seyahat süresi hesaplama
other_2 = other.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})

# Ortalama ve standart sapma hesaplama
mean_tt_4 = other_2['travel_time'].mean()
std_tt_4 = other_2['travel_time'].std()

print("Ortalama travel_time:", mean_tt_4)
print("Standart sapma (std):", std_tt_4)


Ortalama travel_time: 32.194451861195255
Standart sapma (std): 27.271350204428824


### 📊 Tüm Yolculuklar için Ortalama Seyahat Süresi Analizi

Bu adımda, herhangi bir ayrım yapılmaksızın tüm `same_p_and_k_y` gruplarının toplam `travel_time` değerleri incelenmiştir.

### 🧮 Hesaplamalar:
- Her yolculuk grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_10`)
  - **Standart sapma** (`std_tt_10`)
  hesaplanmıştır.

Bu analiz, tüm yolculuk türlerinin genel süresel karakteristiğini ortaya koyarak önceki alt gruplarla yapılacak karşılaştırmalar için temel oluşturur.


In [49]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# Sadece İstanbul ilçelerini filtrele
istanbul_data = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)]

# same_p_and_k_y'ye göre gruplama ve travel_time toplamı
total_2 = istanbul_data.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})

# Ortalama ve standart sapma
mean_tt_10 = total_2['travel_time'].mean()
std_tt_10 = total_2['travel_time'].std()

print("Ortalama travel_time:", mean_tt_10)
print("Standart sapma (std):", std_tt_10)


Ortalama travel_time: 30.76801959254365
Standart sapma (std): 26.152532608567547


### 🚗 Evden İşe Gidiş – Yürüyüş Hariç Ulaşım Modları için Seyahat Süresi Analizi

Bu analizde yalnızca **evden işe** yapılan ve **araç veya toplu taşıma kullanılan** yolculuklar incelenmiştir.  
**Yürüyerek yapılan işe gidişler hariç tutulmuştur.**

### 🎯 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)`
- **Amaç**: `İşe gidiş`
- **Ulaşım Modu**: `Yürüme` dışındaki tüm modlar (`mode != 'Yürüme'`)

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_5`)
  - **Standart sapma** (`std_tt_5`)
  hesaplanmıştır.

Bu analiz, özellikle işe giderken araç ya da toplu taşıma kullanma davranışının zaman maliyetini ortaya koyar.


In [50]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

home_work_vehicle = df_selected_v2[
    (df_selected_v2['orig_loc_type'] == 'EV(SADECE İKAMET EDİLEN EV)') &
    (df_selected_v2['district'].isin(istanbul_ilceleri)) &
    (df_selected_v2['purpose'] == 'İşe gidiş') &
    (df_selected_v2['mode_choice'] != 'Walking')
]

home_work_vehicle_2 = home_work_vehicle.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})

mean_tt_5 = home_work_vehicle_2['travel_time'].mean()
std_tt_5 = home_work_vehicle_2['travel_time'].std()

print("Ortalama travel_time:", mean_tt_5)
print("Standart sapma (std):", std_tt_5)


Ortalama travel_time: 43.14671163575042
Standart sapma (std): 23.965003959718814


### 🎒 Evden Okula Gidiş – Yürüyüş Hariç Ulaşım Modları için Seyahat Süresi Analizi

Bu analizde, **evden okula yapılan** ve **yürüyüş dışındaki** ulaşım modlarının kullanıldığı yolculuklar incelenmiştir.

### 🎯 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)`
- **Amaç**: 
  - `Okula gidiş(ilk ve ortaöğretim)`
  - `Okula gidiş(lise)`
  - `Okula gidiş(üniversite ve üzeri)`
- **Ulaşım Modu**: `Yürüme` hariç tüm modlar (`mode != 'Yürüme'`)

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_6`)
  - **Standart sapma** (`std_tt_6`)
  hesaplanmıştır.

Bu analiz, öğrencilerin **araç, servis ya da toplu taşıma** ile gerçekleştirdikleri okul yolculuklarının zaman maliyetini anlamak açısından önemlidir.


In [51]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]



home_school_vehicle = df_selected_v2[
    (df_selected_v2['orig_loc_type'] == 'EV(SADECE İKAMET EDİLEN EV)') &
    (df_selected_v2['mode_choice'] != 'Walking') &
    (df_selected_v2['district'].isin(istanbul_ilceleri)) &
    (df_selected_v2['purpose'].isin([
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]))
]
home_school_vehicle_2 = home_school_vehicle.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})
mean_tt_6 = home_school_vehicle_2['travel_time'].mean()
std_tt_6 = home_school_vehicle_2['travel_time'].std()
print("Ortalama travel_time:", mean_tt_6)
print("Standart sapma (std):", std_tt_6)

Ortalama travel_time: 41.00076569678407
Standart sapma (std): 22.564900358626446


### 🏡 Evden Diğer Amaçlarla Yapılan Araçlı Yolculukların Seyahat Süresi Analizi

Bu analizde, **evden çıkış noktası** olan ve **yürüyüş harici ulaşım modlarıyla** yapılan çeşitli günlük amaçlı yolculuklar değerlendirilmiştir.

### 🎯 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)`
- **Ulaşım Modu**: `Yürüme` dışındaki tüm modlar (`mode != 'Yürüme'`)
- **Amaç** (purpose): 
  - Akraba/Arkadaş Ziyareti  
  - Çocuk veya diğer kişileri alma/bırakma  
  - Eve dönüş  
  - Sağlık kurumuna gidiş  
  - Alışveriş  
  - İş takibi / ödeme işleri  
  - Spor, eğlence gibi aktiviteler  
  - Yeme/içme gibi sosyal aktiviteler  

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_7`)
  - **Standart sapma** (`std_tt_7`)
  hesaplanmıştır.

Bu analiz, bireylerin günlük yaşam aktivitelerine yönelik araç veya toplu taşıma kullanımıyla ne kadar süre yolculuk yaptıklarını anlamamıza yardımcı olur.


In [52]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]


home_other_vehicle = df_selected_v2[
    (df_selected_v2['orig_loc_type'] == 'EV(SADECE İKAMET EDİLEN EV)') &
    (df_selected_v2['mode_choice'] != 'Walking') &
     (df_selected_v2['district'].isin(istanbul_ilceleri)) &
    (df_selected_v2['purpose'].isin([
    'Akraba/Arkadaş  Ziyareti',
    'Çocuk Alma/Bırakma',
    'Diğer',
    'Diğer Alma/Bırakma',
    'Eve gidiş',
    'Sağlık Kurumuna gidiş',
    'Alışveriş',
    'Diğer',
    'İş takibi/ Ödemeler',
    'Spor, Eğlence vb. gidiş',
    'Yeme/içme Gidiş'
    ]))
]

home_other_vehicle_2 = home_other_vehicle.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})
mean_tt_7 = home_other_vehicle_2['travel_time'].mean()
std_tt_7 = home_other_vehicle_2['travel_time'].std()
print("Ortalama travel_time:", mean_tt_7)
print("Standart sapma (std):", std_tt_7)

Ortalama travel_time: 39.847443895325284
Standart sapma (std): 26.174021134346187


### 🏙️ Ev Dışı Başlangıçlı Araçlı Yolculukların Seyahat Süresi Analizi

Bu analizde, **başlangıç noktası ev olmayan** ve **araç veya toplu taşıma gibi yürüyüş harici ulaşım modlarıyla** yapılan yolculuklar incelenmiştir.

### 🎯 Filtreleme Kriterleri:
- **Başlangıç Lokasyonu**: `EV(SADECE İKAMET EDİLEN EV)` **değil**
- **Ulaşım Modu**: `Yürüme` dışındaki tüm modlar

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` hesaplanmıştır.
- Bu değerlere göre:
  - **Ortalama seyahat süresi** (`mean_tt_8`)
  - **Standart sapma** (`std_tt_8`)
  hesaplanmıştır.

Bu analiz, örneğin iş yerinden veya başka bir sosyal noktadan başlayan çok modlu ya da özel araçla yapılan yolculukların süresel özelliklerini ortaya koymaktadır.


In [53]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]



other_vehicle= df_selected_v2[
    (df_selected_v2['orig_loc_type'] != 'EV(SADECE İKAMET EDİLEN EV)') &
     (df_selected_v2['district'].isin(istanbul_ilceleri)) &
    (df_selected_v2['mode_choice'] != 'Walking')
]

other_vehicle_2= other_vehicle.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})
mean_tt_8 = other_vehicle_2['travel_time'].mean()
std_tt_8 = other_vehicle_2['travel_time'].std()
print("Ortalama travel_time:", mean_tt_8)
print("Standart sapma (std):", std_tt_8)


Ortalama travel_time: 47.60366896603575
Standart sapma (std): 28.626672716350296


### 🚗 Tüm Araçlı Yolculukların Seyahat Süresi Analizi

Bu bölümde, **yürüyüş dışındaki tüm ulaşım modlarıyla** (özel araç, toplu taşıma, servis, mikromobilite vb.) gerçekleştirilen yolculuklar analiz edilmiştir.

### 🎯 Filtreleme Kriteri:
- `mode` ≠ `Yürüme`

### 🧮 Hesaplamalar:
- Her `same_p_and_k_y` grubu için toplam `travel_time` değerleri alınmıştır.
- Bu gruplara göre:
  - **Ortalama seyahat süresi** (`mean_tt_9`)
  - **Standart sapma** (`std_tt_9`)
  hesaplanmıştır.

Bu analiz, kentsel hareketlilikte araç kullanımının süre bazlı etkisini ortaya koymayı amaçlamaktadır.


In [54]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

Vehicle_total = df_selected_v2[
    (df_selected_v2['mode_choice'] != 'Walking') &
    (df_selected_v2['district'].isin(istanbul_ilceleri))
]

Vehicle_total_2 = Vehicle_total.groupby(["same_p_and_k_y"], as_index=False).agg({
    "travel_time": "sum",
})

mean_tt_9 = Vehicle_total_2['travel_time'].mean()
std_tt_9 = Vehicle_total_2['travel_time'].std()

print("Ortalama travel_time:", mean_tt_9)
print("Standart sapma (std):", std_tt_9)


Ortalama travel_time: 45.858497356122186
Standart sapma (std): 27.583070639643157


 <span style="font-size:22pt; color:red;"><b>Figure 151. Leisure Active Mobility Frequency & Travel Times of Different Person Types (İstanbul SUMP Stage II HTS)  </b></span>

### 

Bu bölümde, aşağıdaki amaçlarla yapılan **sosyal ve eğlence odaklı yolculuklar** analiz edilmiştir:

- Akraba/Arkadaş Ziyareti  
- Alışveriş  
- Diğer  
- İş Takibi / Ödemeler  
- Spor, Eğlence vb. Gidiş  
- Yeme/İçme Gidiş
- Her `same_p_and_k_y` grubu için süre bileşenleri (`travel_time`, `vehicle_time`, `access_time`, `egress_time`, `waiting_time`) toplanmıştır.
- Bu gruplar **Employment Group**'a göre sınıflandırılmış ve ortalama değerleri hesaplanmıştır.

Bu analiz, farklı iş/istihdam gruplarının sosyal ve eğlence temelli yolculuk alışkanlıklarını süre bileşenleri açısından karşılaştırmalı olarak göstermektedir.


In [55]:

purpo = [
    "Akraba/Arkadaş  Ziyareti",
    "Alışveriş",
    "Diğer",
    "İş takibi/ Ödemeler",
    "Spor, Eğlence vb. gidiş",
    "Yeme/içme Gidiş"
]

df_gidis = df_selected_v2[df_selected_v2["purpose"].isin(purpo)]
leisure = df_gidis.groupby(["same_p_and_k_y", "Employment Group"], as_index=False).agg({
    "travel_time": "sum",
    "p_id": "first",
    "id": "first",
    "purpose" : "first"
})
leisure['purpose'].unique()
# mode'a göre grupla ve ortalamaları al
lesiure_2 = leisure.groupby("Employment Group")[[
    "travel_time"
]].mean().reset_index()

# Sonucu yazdır
lesiure_2

,Employment Group,travel_time
0,65+ Retired,19.394029
1,Adult Student,28.835152
2,Adult Unemployed,22.497028
3,Child,23.143939
4,Child Student,21.864476
5,Full Time,26.368474
6,Other,15.977349
7,Part Time,31.213873


 <span style="font-size:22pt; color:red;"><b>Figure 109. Travel Time, In Vehicle Time, Waiting, Access, and Egress Times in Public Transportation for Work Purpose Trips by Destination District in Istanbul (İstanbul SUMP Stage II HTS) </b></span>
#### 
Bu analiz, **toplu taşıma** kullanarak **"İşe Gidiş"** amacıyla yapılan yolculuklarda, her yolculuk grubunun **son varış ilçesine (`dest_dist`)** göre süre bileşenlerinin ortalamalarını sunmaktadır.

 📌 **Analiz Adımları**:
1. `id` değeri kullanılarak her yolculuk grubunun (same_p_and_k_y) **son satırı (final destination)** belirlenmiştir.
2. Yalnızca `mode_category = Public Transport` ve `purpose = İşe gidiş` olan satırlar filtrelenmiştir.
3. Bu satırlar her yolculuk grubu için gruplanarak **toplam süre bileşenleri** hesaplanmıştır:
   - `travel_time`, `vehicle_time`, `access_time`, `egress_time`, `waiting_time`
4. Gruplar, belirlenen `final_dest` (son varış ilçesi) ile eşleştirilmiş ve ilçelere göre ortalama süre değerleri çıkarılmıştır.



In [56]:
import pandas as pd

# 0. id'yi sayıya çevir
df_selected_v2['id_int'] = (
    df_selected_v2['id']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(int)
)

# 1. Final destinasyonu belirle: her grup için en yüksek id’li satırın dest_dist değeri
last_ids = df_selected_v2.loc[
    df_selected_v2.groupby("same_p_and_k_y")["id_int"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Sadece Public Transport ve İşe Gidiş olanları filtrele
df_pt = df_selected_v2[
    (df_selected_v2["mode_category"] == "Public Transport") &
    (df_selected_v2["purpose"] == "İşe gidiş")
]

# 3. Özel zaman hesaplama fonksiyonu
def custom_agg(group):
    travel_time_sum = group['travel_time'].sum()
    vehicle_time_sum = group['vehicle_time'].sum()
    waiting_time_sum = group['waiting_time'].sum()
    
    access_time_value = group.loc[group['id_int'].idxmin(), 'access_time']
    egress_time_value = group.loc[group['id_int'].idxmax(), 'egress_time']
    
    group_sorted = group.sort_values(by='id_int')
    n = len(group_sorted)
    transfer_time_value = group_sorted.iloc[:-1]['egress_time'].sum() if n > 1 else 0
    
    return pd.Series({
        'travel_time': travel_time_sum,
        'vehicle_time': vehicle_time_sum,
        'waiting_time': waiting_time_sum,
        'access_time': access_time_value,
        'egress_time': egress_time_value,
        'transfer_time': transfer_time_value
    })

# 4. Gruplama ve hesaplama
agg_df = (
    df_pt
    .groupby("same_p_and_k_y")
    .apply(custom_agg)
    .reset_index()
)

# 5. travel_time bileşenlerini kontrol et
agg_df['difference'] = (
    agg_df['travel_time']
    - agg_df['vehicle_time']
    - agg_df['waiting_time']
    - agg_df['access_time']
    - agg_df['egress_time']
    - agg_df['transfer_time']
)

# 6. Fark varsa vehicle_time’a ekle
agg_df.loc[agg_df['difference'] > 0, 'vehicle_time'] += agg_df['difference']
agg_df.drop(columns='difference', inplace=True)

# 7. Final destinasyonu ile birleştir
agg_df = agg_df.merge(last_ids, on="same_p_and_k_y", how="left")

# 8. Ortalama süreleri destinasyona göre hesapla
work_mean = agg_df.groupby("final_dest")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean().reset_index()
work_mean.head(1)

C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\3063094530.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2['id_int'] = (


,final_dest,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
0,ADALAR,67.0,42.0,8.333333,4.666667,8.666667,3.333333


### 🏙️ İstanbul İlçelerine Yapılan İşe Gidiş Amaçlı Toplu Taşıma Yolculuklarının Ortalama Süreleri

Bu bölümde, **İstanbul il sınırları içinde** işe gitmek amacıyla **toplu taşıma** kullanan yolculuk gruplarının ortalama süre bileşenleri incelenmektedir.

### 🔍 Filtreleme Kriteri:
- `final_dest` ilçesi, İstanbul’un 39 resmi ilçesinden biri olmalıdır.

### 📈 Hesaplanan Ortalama Süre Bileşenleri:
- **Toplam Yolculuk Süresi (`travel_time`)**
- **Araç İçinde Geçen Süre (`vehicle_time`)**
- **Toplu Taşımaya Ulaşım Süresi (`access_time`)**
- **İniş Sonrası Hedefe Yürüme Süresi (`egress_time`)**
- **Bekleme Süresi (`waiting_time`)**

Bu analiz, İstanbul içi işe gidişlerde farklı ulaşım aşamalarının yolculuk süresine ne kadar katkı sağladığını anlamamıza yardımcı olur.


In [57]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçelerine filtreleme
agg_df_filtered = agg_df[agg_df['final_dest'].isin(istanbul_ilceleri)].copy()

# İlçe bazlı ortalamaları hesapla
work_mean = agg_df_filtered.groupby("final_dest")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean().reset_index()

# Genel ortalamayı hesapla
general_mean = pd.DataFrame(agg_df_filtered[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean()).T
general_mean.insert(0, "final_dest", "GENEL ORTALAMA")

# Hepsini birleştir
final_output = pd.concat([work_mean, general_mean], ignore_index=True)
# Sadece GENEL ORTALAMA satırını göster
genel_ortalama = final_output[final_output['final_dest'] == "GENEL ORTALAMA"].copy()
genel_ortalama.loc[:, genel_ortalama.columns[1:]] = genel_ortalama.loc[:, genel_ortalama.columns[1:]].round(1)
genel_ortalama

,final_dest,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
39,GENEL ORTALAMA,52.8,32.4,6.8,6.8,6.4,0.4


### 🏙️ Kocaeli ve Tekirdağ Yapılan İşe Gidiş Amaçlı Toplu Taşıma Yolculuklarının Ortalama Süreleri


In [58]:
import pandas as pd

# Kocaeli ilçeleri listesi
kocaeli_ilceleri = ["ÇAYIROVA", "GEBZE", "DARICA"]

# Kocaeli ilçelerine filtreleme
agg_df_filtered = agg_df[agg_df['final_dest'].isin(kocaeli_ilceleri)].copy()

# İlçe bazlı ortalamaları hesapla
work_mean = agg_df_filtered.groupby("final_dest")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean().reset_index()

# Genel ortalamayı hesapla
general_mean = pd.DataFrame(agg_df_filtered[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean()).T
general_mean.insert(0, "final_dest", "GENEL ORTALAMA")

# Hepsini birleştir
final_output = pd.concat([work_mean, general_mean], ignore_index=True)

# Sadece GENEL ORTALAMA satırını göster, 1 ondalık basamakla
genel_ortalama = final_output[final_output['final_dest'] == "GENEL ORTALAMA"].copy()
genel_ortalama.loc[:, genel_ortalama.columns[1:]] = genel_ortalama.loc[:, genel_ortalama.columns[1:]].round(1)

genel_ortalama


,final_dest,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
3,GENEL ORTALAMA,39.1,20.7,6.1,6.7,5.5,0.2


In [59]:

tekirdag_ilceleri = [
    "SARAY","ÇERKEZKÖY","ÇORLU","KAPAKLI"
]

# Kocaeli ilçelerine filtreleme
agg_df_filtered = agg_df[agg_df['final_dest'].isin(tekirdag_ilceleri)].copy()

# İlçe bazlı ortalamaları hesapla
work_mean = agg_df_filtered.groupby("final_dest")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean().reset_index()

# Genel ortalamayı hesapla
general_mean = pd.DataFrame(agg_df_filtered[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean()).T
general_mean.insert(0, "final_dest", "GENEL ORTALAMA")

# Hepsini birleştir
final_output = pd.concat([work_mean, general_mean], ignore_index=True)

# Sadece GENEL ORTALAMA satırını göster, 1 ondalık basamakla
genel_ortalama = final_output[final_output['final_dest'] == "GENEL ORTALAMA"].copy()
genel_ortalama.loc[:, genel_ortalama.columns[1:]] = genel_ortalama.loc[:, genel_ortalama.columns[1:]].round(1)

genel_ortalama


,final_dest,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
4,GENEL ORTALAMA,40.0,24.9,5.0,5.1,5.0,0.0


<span style="font-size:22pt; color:red;"><b>Travel Time & Access/Egress in PT Usage of University Students in School Location</b></span>

<ul style="font-size:18pt; color:red; font-weight:bold;"> <li>Table 29: By Destination District in İstanbul (SUMP Stage II HTS)</li> <li>Figure 131: By Destination District in Kocaeli</li> <li>Figure 132: By Destination District in Tekirdağ</li> </ul>

### 
Bu analizde, **üniversite ve üzeri eğitim** amacıyla gerçekleştirilen **toplu taşıma yolculukları** incelenmiştir. Yolculukların son varış noktası (`final_dest`) baz alınarak, süre bileşenlerinin ortalamaları hesaplanmıştır.

- `mode_category` = **Public Transport**
- `purpose` = **Okula gidiş (üniversite ve üzeri)**

- - **Toplam Yolculuk Süresi (`travel_time`)**
- **Araçta Geçirilen Süre (`vehicle_time`)**
- **Ulaşıma Harcanan Süre (`access_time`)**
- **Ara Duraklardan Varışa Yürüme Süresi (`egress_time`)**
- **Toplu Taşıma Bekleme Süresi (`waiting_time`)*

In [60]:
import pandas as pd

# 0. id'yi sayıya çevir
df_selected_v2['id_int'] = (
    df_selected_v2['id']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(int)
)

# 1. Final destinasyonu belirle: her grup için en yüksek id’li satırın dest_dist değeri
last_ids = df_selected_v2.loc[
    df_selected_v2.groupby("same_p_and_k_y")["id_int"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Sadece Public Transport ve Okula gidiş (üniversite ve üzeri) olanları filtrele
df_pt = df_selected_v2[
    (df_selected_v2["mode_category"] == "Public Transport") &
    (df_selected_v2["purpose"] == "Okula gidiş(üniversite ve üzeri)")
]

# 3. Özel zaman hesaplama fonksiyonu
def custom_agg(group):
    travel_time_sum = group['travel_time'].sum()
    vehicle_time_sum = group['vehicle_time'].sum()
    waiting_time_sum = group['waiting_time'].sum()

    access_time_value = group.loc[group['id_int'].idxmin(), 'access_time']
    egress_time_value = group.loc[group['id_int'].idxmax(), 'egress_time']

    group_sorted = group.sort_values(by='id_int')
    n = len(group_sorted)
    transfer_time_value = group_sorted.iloc[:-1]['egress_time'].sum() if n > 1 else 0

    return pd.Series({
        'travel_time': travel_time_sum,
        'vehicle_time': vehicle_time_sum,
        'waiting_time': waiting_time_sum,
        'access_time': access_time_value,
        'egress_time': egress_time_value,
        'transfer_time': transfer_time_value
    })

# 4. Gruplama ve hesaplama
agg_df = (
    df_pt
    .groupby("same_p_and_k_y")
    .apply(custom_agg)
    .reset_index()
)

# 5. travel_time bileşenlerini kontrol et
agg_df['difference'] = (
    agg_df['travel_time']
    - agg_df['vehicle_time']
    - agg_df['waiting_time']
    - agg_df['access_time']
    - agg_df['egress_time']
    - agg_df['transfer_time']
)

# 6. Fark varsa vehicle_time’a ekle
agg_df.loc[agg_df['difference'] > 0, 'vehicle_time'] += agg_df['difference']
agg_df.drop(columns='difference', inplace=True)

# 7. Final destinasyonu ile birleştir
agg_df = agg_df.merge(last_ids, on="same_p_and_k_y", how="left")

# 8. Ortalama süreleri destinasyona göre hesapla
univ_mean = agg_df.groupby("final_dest")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean().reset_index()

# 9. İlk satırı göster
univ_mean.head(1)


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\614951146.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2['id_int'] = (


,final_dest,travel_time,vehicle_time,access_time,egress_time,waiting_time,transfer_time
0,ARNAVUTKÖY,86.2,70.4,6.6,6.0,3.2,0.0


In [61]:
import pandas as pd

# 0. id'yi sayıya çevir
df_selected_v2['id_int'] = (
    df_selected_v2['id']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(int)
)

# 1. Final destinasyonu belirle: her grup için en yüksek id’li satırın dest_dist değeri
last_ids = df_selected_v2.loc[
    df_selected_v2.groupby("same_p_and_k_y")["id_int"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Sadece Public Transport ve Okula gidiş (üniversite ve üzeri) olanları filtrele
df_pt = df_selected_v2[
    (df_selected_v2["mode_category"] == "Public Transport") &
    (df_selected_v2["purpose"] == "Okula gidiş(üniversite ve üzeri)")
]

# 3. Özel zaman hesaplama fonksiyonu
def custom_agg(group):
    travel_time_sum = group['travel_time'].sum()
    vehicle_time_sum = group['vehicle_time'].sum()
    waiting_time_sum = group['waiting_time'].sum()

    access_time_value = group.loc[group['id_int'].idxmin(), 'access_time']
    egress_time_value = group.loc[group['id_int'].idxmax(), 'egress_time']

    group_sorted = group.sort_values(by='id_int')
    n = len(group_sorted)
    transfer_time_value = group_sorted.iloc[:-1]['egress_time'].sum() if n > 1 else 0

    return pd.Series({
        'travel_time': travel_time_sum,
        'vehicle_time': vehicle_time_sum,
        'waiting_time': waiting_time_sum,
        'access_time': access_time_value,
        'egress_time': egress_time_value,
        'transfer_time': transfer_time_value
    })

# 4. Gruplama ve hesaplama
agg_df = (
    df_pt
    .groupby("same_p_and_k_y")
    .apply(custom_agg)
    .reset_index()
)

# 5. travel_time bileşenlerini kontrol et
agg_df['difference'] = (
    agg_df['travel_time']
    - agg_df['vehicle_time']
    - agg_df['waiting_time']
    - agg_df['access_time']
    - agg_df['egress_time']
    - agg_df['transfer_time']
)

# 6. Fark varsa vehicle_time’a ekle
agg_df.loc[agg_df['difference'] > 0, 'vehicle_time'] += agg_df['difference']
agg_df.drop(columns='difference', inplace=True)

# 7. Final destinasyonu ile birleştir
agg_df = agg_df.merge(last_ids, on="same_p_and_k_y", how="left")

# 8. İstanbul ilçelerine göre filtrele
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU", "ÇERKEZKÖY", "ÇORLU", "ERGENE", "KAPAKLI", "MARMARAEREĞLİSİ", "SARAY","ÇAYIROVA", "DARICA", "GEBZE"
]
agg_df = agg_df[agg_df["final_dest"].isin(istanbul_ilceleri)]

# 9. Ortalama süreleri destinasyona göre hesapla
univ_mean = agg_df.groupby("final_dest")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time",
    "transfer_time"
]].mean().reset_index()

# 10. Genel ortalamayı hesapla ve tabloya ekle
genel_ortalama = univ_mean.drop(columns="final_dest").mean(numeric_only=True)
genel_ortalama["final_dest"] = "Genel Ortalama"
univ_mean = pd.concat([univ_mean, pd.DataFrame([genel_ortalama])], ignore_index=True)

print(univ_mean[univ_mean["final_dest"] == "Genel Ortalama"])

C:\Users\Paraboly\AppData\Local\Temp\ipykernel_28416\4204349974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_v2['id_int'] = (


        final_dest  travel_time  vehicle_time  access_time  egress_time  \
43  Genel Ortalama    60.619981     38.932401     7.688284     6.610344   

    waiting_time  transfer_time  
43      6.862585       0.526367  


 <span style="font-size:22pt; color:red;"><b>Table 26. Changes in Average Travel Times for All Trips and Vehicle Trips for 2006, 2012 and 2024 -PEDESTRIAN</b></span>

### 

In [62]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçeleriyle filtrelenmiş versiyonu
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# 0. id'yi sayıya çevir
df_filtered["id"] = df_filtered["id"].astype(str).str.replace(",", "").astype(int)

# 1. Her grup için en yüksek id’ye sahip satırdan dest_dist bilgisini al
final_destinations = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Her grup için en düşük id’ye sahip satırdan orig_dist bilgisini al
initial_origins = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmin(),
    ["same_p_and_k_y", "orig_dist"]
].rename(columns={"orig_dist": "initial_orig"})

# 3. Sadece Walking moduna sahip satırları al
pt_school_trips = df_filtered[
    (df_filtered["mode_category"] == "Walking")
]

# 4. Bu satırları same_p_and_k_y’ye göre grupla ve süreleri topla
aggregated_durations = pt_school_trips.groupby("same_p_and_k_y").agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "p_id": "first"
}).reset_index()

# 5. Final destinasyon ile birleştir
merged_with_dest = aggregated_durations.merge(final_destinations, on="same_p_and_k_y", how="left")

# 6. İlk origin mesafesi ile birleştir
final_merged_result = merged_with_dest.merge(initial_origins, on="same_p_and_k_y", how="left")


In [63]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
final_merged_result['orig_continent'] = final_merged_result['initial_orig'].map(continent_mapping)
final_merged_result['dest_continent'] = final_merged_result['final_dest'].map(continent_mapping)

In [64]:
final_merged_result["orig_continent"].unique()

array(['Europe', 'Asia'], dtype=object)

In [65]:
final_merged_result["dest_continent"].unique()

array(['Europe', 'Asia'], dtype=object)

In [66]:
final_merged_result["continental_type"] = final_merged_result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [67]:
final_merged_result["continental_type"].unique()

array(['Within Europe', 'Within Asia', 'Intercontinental'], dtype=object)

In [68]:
# Intercontinental olmayan verileri filtrele
filtered = final_merged_result[final_merged_result['continental_type'] != "Intercontinental"]

# Ortalama ve standart sapmayı continental_type bazında hesapla
continental = filtered.groupby("continental_type")[["travel_time"]].agg(['mean', 'std']).reset_index()
continental.columns = ['continental_type', 'travel_time_mean', 'travel_time_std']

# Genel ortalama ve std hesapla
general_mean = filtered["travel_time"].mean()
general_std = filtered["travel_time"].std()

# Genel satırı oluştur
general_row = pd.DataFrame([{
    'continental_type': 'GENEL',
    'travel_time_mean': general_mean,
    'travel_time_std': general_std
}])

# Genel satırı ekle
continental_with_total = pd.concat([continental, general_row], ignore_index=True)

# İsteğe bağlı: Virgülden sonra 1 basamak
continental_with_total = continental_with_total.round(1)

# İlk 3 + GENEL satırını göster
continental_with_total


,continental_type,travel_time_mean,travel_time_std
0,Within Asia,17.1,16.3
1,Within Europe,17.5,14.7
2,GENEL,17.4,15.3


 <span style="font-size:22pt; color:red;"><b>Table 26. Changes in Average Travel Times for All Trips and Vehicle Trips for 2006, 2012 and 2024 -VEHİCLE </b></span>

### 

In [ ]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# Sadece İstanbul ilçelerindeki veriler
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# 0. id'yi sayıya çevir
df_filtered["id"] = df_filtered["id"].astype(str).str.replace(",", "").astype(int)

# 1. Her grup için en yüksek id’ye sahip satırdan dest_dist bilgisini al
final_destinations = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Her grup için en düşük id’ye sahip satırdan orig_dist bilgisini al
initial_origins = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmin(),
    ["same_p_and_k_y", "orig_dist"]
].rename(columns={"orig_dist": "initial_orig"})

# 3. Sadece Walking olmayan modlara sahip satırları al
pt_school_trips = df_filtered[
    (df_filtered["mode_category"] != "Walking")
]

# 4. Bu satırları same_p_and_k_y’ye göre grupla ve süreleri topla
aggregated_durations = pt_school_trips.groupby("same_p_and_k_y").agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "p_id": "first"
}).reset_index()

# 5. Final destinasyon ile birleştir
merged_with_dest = aggregated_durations.merge(final_destinations, on="same_p_and_k_y", how="left")

# 6. İlk origin mesafesi ile birleştir
final_merged_result = merged_with_dest.merge(initial_origins, on="same_p_and_k_y", how="left")


In [ ]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
final_merged_result['orig_continent'] = final_merged_result['initial_orig'].map(continent_mapping)
final_merged_result['dest_continent'] = final_merged_result['final_dest'].map(continent_mapping)

In [ ]:
final_merged_result["continental_type"] = final_merged_result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [ ]:
final_merged_result["continental_type"].unique()

In [ ]:
# Ortalama ve standart sapmayı hesapla
continental = final_merged_result.groupby("continental_type")[[
    "travel_time"
]].agg(['mean', 'std']).reset_index()

# Kolon isimlerini düzelt
continental.columns = ['continental_type', 'travel_time_mean', 'travel_time_std']

# Genel ortalama ve standart sapmayı hesapla
general_mean = final_merged_result["travel_time"].mean()
general_std = final_merged_result["travel_time"].std()

# Genel satırını oluştur
general_row = pd.DataFrame([{
    'continental_type': 'GENEL',
    'travel_time_mean': general_mean,
    'travel_time_std': general_std
}])

# Birleştir
continental_with_total = pd.concat([continental, general_row], ignore_index=True)

# Virgülden sonra 1 basamakla göster
continental_with_total = continental_with_total.round(1)

# Sonuç
continental_with_total


 <span style="font-size:22pt; color:red;"><b>Table 26. Changes in Average Travel Times for All Trips and Vehicle Trips for 2006, 2012 and 2024 -ALL </b></span>

### 

In [ ]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# Sadece İstanbul ilçelerini filtrele
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# 0. id'yi sayıya çevir
df_filtered["id"] = df_filtered["id"].astype(str).str.replace(",", "").astype(int)

# 1. Her grup için en yüksek id’ye sahip satırdan dest_dist bilgisini al
final_destinations = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Her grup için en düşük id’ye sahip satırdan orig_dist bilgisini al
initial_origins = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmin(),
    ["same_p_and_k_y", "orig_dist"]
].rename(columns={"orig_dist": "initial_orig"})

# 3. Tüm satırları kullan (Walking filtresi yok)
pt_school_trips = df_filtered

# 4. same_p_and_k_y’ye göre gruplama ve süreleri toplama
aggregated_durations = pt_school_trips.groupby("same_p_and_k_y").agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "mode_category":"first",
    "p_id": "first"
}).reset_index()

# 5. Final destinasyonla birleştir
merged_with_dest = aggregated_durations.merge(final_destinations, on="same_p_and_k_y", how="left")

# 6. İlk origin mesafesiyle birleştir
final_merged_result = merged_with_dest.merge(initial_origins, on="same_p_and_k_y", how="left")


In [ ]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
final_merged_result['orig_continent'] = final_merged_result['initial_orig'].map(continent_mapping)
final_merged_result['dest_continent'] = final_merged_result['final_dest'].map(continent_mapping)

In [ ]:
final_merged_result["continental_type"] = final_merged_result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [ ]:
final_merged_result["continental_type"].unique()

In [ ]:
import pandas as pd

# Walking modundaki Intercontinental verileri hariç tut
filtered = final_merged_result[
    ~((final_merged_result['mode_category'] == 'Walking') & 
      (final_merged_result['continental_type'] == 'Intercontinental'))
]

# 1. Kıta bazlı ortalama ve standart sapma hesapla
continental = filtered.groupby("continental_type")["travel_time"].agg(
    travel_time_mean="mean",
    travel_time_std="std"
).reset_index()

# 2. Genel ortalama ve standart sapma hesapla
general_mean = filtered["travel_time"].mean()
general_std = filtered["travel_time"].std()

# 3. Genel satırını oluştur
general_row = pd.DataFrame([{
    'continental_type': 'GENEL',
    'travel_time_mean': general_mean,
    'travel_time_std': general_std
}])

# 4. Var olan tablo ile genel satırı birleştir
continental_with_total = pd.concat([continental, general_row], ignore_index=True)

# 5. "Intercontinental" satırını hariç tut
continental_with_total = continental_with_total[continental_with_total['continental_type'] != 'Intercontinental']

# 6. Sayıları virgülden sonra 1 basamağa yuvarla
continental_with_total[["travel_time_mean", "travel_time_std"]] = continental_with_total[[
    "travel_time_mean", "travel_time_std"
]].round(1)

# 7. Sonuç
continental_with_total


 <span style="font-size:22pt; color:red;"><b>Table 26. Changes in Average Travel Times for All Trips and Vehicle Trips for 2006, 2012 and 2024 -PRİVATE VEHİCLE </b></span>

### 

In [ ]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# Sadece İstanbul ilçeleriyle sınırlı veri
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# 0. id'yi sayıya çevir
df_filtered["id"] = df_filtered["id"].astype(str).str.replace(",", "").astype(int)

# 1. En yüksek id’ye sahip satırdan final destination
final_destinations = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. En düşük id’ye sahip satırdan initial origin
initial_origins = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmin(),
    ["same_p_and_k_y", "orig_dist"]
].rename(columns={"orig_dist": "initial_orig"})

# 3. Sadece Private Vehicle moduna sahip satırları al
pt_school_trips = df_filtered[
    (df_filtered["mode_category"] == "Private Vehicle")
]

# 4. same_p_and_k_y’ye göre grupla, süreleri topla
aggregated_durations = pt_school_trips.groupby("same_p_and_k_y").agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "p_id": "first"
}).reset_index()

# 5. Final destinasyonla birleştir
merged_with_dest = aggregated_durations.merge(final_destinations, on="same_p_and_k_y", how="left")

# 6. İlk origin mesafesi ile birleştir
final_merged_result = merged_with_dest.merge(initial_origins, on="same_p_and_k_y", how="left")


In [ ]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
final_merged_result['orig_continent'] = final_merged_result['initial_orig'].map(continent_mapping)
final_merged_result['dest_continent'] = final_merged_result['final_dest'].map(continent_mapping)

In [ ]:
final_merged_result["continental_type"] = final_merged_result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [ ]:
final_merged_result["continental_type"].unique()

In [ ]:
import pandas as pd

# Kıta bazında ortalama ve std hesapla
continental = final_merged_result.groupby("continental_type")["travel_time"].agg(
    travel_time_mean="mean",
    travel_time_std="std"
).reset_index()

# Genel ortalama ve std hesapla
general_mean = final_merged_result["travel_time"].mean()
general_std = final_merged_result["travel_time"].std()

# Genel satırı oluştur
general_row = pd.DataFrame([{
    'continental_type': 'GENEL',
    'travel_time_mean': general_mean,
    'travel_time_std': general_std
}])

# Kıta bazlı tablo ile genel satırı birleştir
continental_with_total = pd.concat([continental, general_row], ignore_index=True)

# Sayıları yuvarla (isteğe bağlı)
continental_with_total[["travel_time_mean", "travel_time_std"]] = continental_with_total[[
    "travel_time_mean", "travel_time_std"
]].round(1)

continental_with_total.head(4)


 <span style="font-size:22pt; color:red;"><b>Table 26. Changes in Average Travel Times for All Trips and Vehicle Trips for 2006, 2012 and 2024 -PUBLIC TRANSPORT </b></span>

### 

In [ ]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçeleri ile filtreleme
df_filtered = df_selected_v2[df_selected_v2["district"].isin(istanbul_ilceleri)].copy()

# 0. id'yi sayıya çevir
df_filtered["id"] = df_filtered["id"].astype(str).str.replace(",", "").astype(int)

# 1. Her grup için en yüksek id’ye sahip satırdan dest_dist bilgisini al
final_destinations = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. Her grup için en düşük id’ye sahip satırdan orig_dist bilgisini al
initial_origins = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmin(),
    ["same_p_and_k_y", "orig_dist"]
].rename(columns={"orig_dist": "initial_orig"})

# 3. Sadece toplu taşıma (Public Transport) moduna sahip satırları al
pt_school_trips = df_filtered[
    (df_filtered["mode_category"] == "Public Transport")
]

# 4. same_p_and_k_y’ye göre grupla ve süreleri topla
aggregated_durations = pt_school_trips.groupby("same_p_and_k_y").agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "p_id": "first"
}).reset_index()

# 5. Final destinasyon ile birleştir
merged_with_dest = aggregated_durations.merge(final_destinations, on="same_p_and_k_y", how="left")

# 6. İlk origin mesafesi ile birleştir
final_merged_result = merged_with_dest.merge(initial_origins, on="same_p_and_k_y", how="left")


In [ ]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
final_merged_result['orig_continent'] = final_merged_result['initial_orig'].map(continent_mapping)
final_merged_result['dest_continent'] = final_merged_result['final_dest'].map(continent_mapping)

In [ ]:
final_merged_result["continental_type"] = final_merged_result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [ ]:
import pandas as pd

# Kıta bazında ortalama ve std hesapla
continental = final_merged_result.groupby("continental_type")["travel_time"].agg(
    travel_time_mean="mean",
    travel_time_std="std"
).reset_index()

# Genel ortalama ve std hesapla
general_mean = final_merged_result["travel_time"].mean()
general_std = final_merged_result["travel_time"].std()

# Genel satırı oluştur
general_row = pd.DataFrame([{
    'continental_type': 'GENEL',
    'travel_time_mean': general_mean,
    'travel_time_std': general_std
}])

# Kıta bazlı tablo ile genel satırı birleştir
continental_with_total = pd.concat([continental, general_row], ignore_index=True)

# Sayıları yuvarla (isteğe bağlı)
continental_with_total[["travel_time_mean", "travel_time_std"]] = continental_with_total[[
    "travel_time_mean", "travel_time_std"
]].round(1)

continental_with_total.head(4)


 <span style="font-size:22pt; color:red;"><b>Table 26. Changes in Average Travel Times for All Trips and Vehicle Trips for 2006, 2012 and 2024 -PUBLIC TRANSPORT & PRİVATE VEHİCLE </b></span>
 
### 

In [ ]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçelerine ait veriyi filtrele
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# 0. id'yi sayıya çevir
df_filtered["id"] = df_filtered["id"].astype(str).str.replace(",", "").astype(int)

# 1. En yüksek id’ye sahip satırdan dest_dist bilgisi
final_destinations = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmax(),
    ["same_p_and_k_y", "dest_dist"]
].rename(columns={"dest_dist": "final_dest"})

# 2. En düşük id’ye sahip satırdan orig_dist bilgisi
initial_origins = df_filtered.loc[
    df_filtered.groupby("same_p_and_k_y")["id"].idxmin(),
    ["same_p_and_k_y", "orig_dist"]
].rename(columns={"orig_dist": "initial_orig"})

# 3. Public Transport veya Private Vehicle olan satırları filtrele
pt_school_trips = df_filtered[
    (df_filtered["mode_category"] == "Public Transport") |
    (df_filtered["mode_category"] == "Private Vehicle")
]

# 4. Grupla ve süreleri topla
aggregated_durations = pt_school_trips.groupby("same_p_and_k_y").agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "p_id": "first"
}).reset_index()

# 5. Final destinasyon ile birleştir
merged_with_dest = aggregated_durations.merge(final_destinations, on="same_p_and_k_y", how="left")

# 6. İlk origin mesafesi ile birleştir
final_merged_result = merged_with_dest.merge(initial_origins, on="same_p_and_k_y", how="left")


In [ ]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
final_merged_result['orig_continent'] = final_merged_result['initial_orig'].map(continent_mapping)
final_merged_result['dest_continent'] = final_merged_result['final_dest'].map(continent_mapping)

In [ ]:
final_merged_result["continental_type"] = final_merged_result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [ ]:
import pandas as pd

# Kıta bazında ortalama ve std hesapla
continental = final_merged_result.groupby("continental_type")["travel_time"].agg(
    travel_time_mean="mean",
    travel_time_std="std"
).reset_index()

# Genel ortalama ve std hesapla
general_mean = final_merged_result["travel_time"].mean()
general_std = final_merged_result["travel_time"].std()

# Genel satırı oluştur
general_row = pd.DataFrame([{
    'continental_type': 'GENEL',
    'travel_time_mean': general_mean,
    'travel_time_std': general_std
}])

# Kıta bazlı tablo ile genel satırı birleştir
continental_with_total = pd.concat([continental, general_row], ignore_index=True)

# Sayıları yuvarla (isteğe bağlı)
continental_with_total[["travel_time_mean", "travel_time_std"]] = continental_with_total[[
    "travel_time_mean", "travel_time_std"
]].round(1)

continental_with_total.head(4)


  <span style="font-size:22pt; color:red;"><b>Table x. Distribution of Average Travel Times by Transportation Modes Across Istanbul Province</b></span>

###  
🚶‍♂🚗🚆 Ulaşım Modlarına Göre Seyahat Süresi Analizi

Aşağıdaki tablo, her bir ulaşım modunun toplam seyahat süresine dair özet istatistikleri sunmaktadır:

- **Ortalama Toplam Seyahat Süresi** (dakika): Aynı grup içerisindeki tüm etapların süresi toplamının ortalaması  
- **Standart Sapma**: Seyahat sürelerinin ne kadar değişkenlik gösterdiğini belirtir  
- **Grup Sayısı**: Her bir mod için hesaplamaya dahil edilen benzersiz grup (same_p_and_k_y) sayısı



In [ ]:
import pandas as pd

# İstanbul ilçeleri listesi
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçelerine göre filtrelenmiş veri
df_filtered = df_selected_v2[df_selected_v2['district'].isin(istanbul_ilceleri)].copy()

# Sonuçları saklamak için boş liste
summary_stats = []

# Benzersiz mode değerleri için döngü
for mode_value in df_filtered['mode'].unique():
    # 1. O moda ait gruplar
    group_ids = df_filtered[df_filtered['mode'] == mode_value]['same_p_and_k_y'].unique()

    # 2. Bu gruplardaki tüm satırları filtrele
    related_rows = df_filtered[df_filtered['same_p_and_k_y'].isin(group_ids)]

    # 3. Grup bazlı travel_time toplamı
    grouped = related_rows.groupby('same_p_and_k_y')['travel_time'].sum().reset_index()

    # 4. Ortalama ve standart sapma
    mean_tt = grouped['travel_time'].mean()
    std_tt = grouped['travel_time'].std()

    # 5. Sonuçları listeye ekle
    summary_stats.append({
        'mode': mode_value,
        'mean_total_travel_time': mean_tt,
        'std_total_travel_time': std_tt,
        'group_count': len(grouped)
    })

# Listeyi DataFrame'e çevir
summary_df = pd.DataFrame(summary_stats)
summary_df


<span style="font-size:22pt; color:red;"><b>Figure 98. Mode choice of public transport (% distribution of among all modes) (İstanbul SUMP Stage II HTS)</b></span>

### 

In [ ]:
# 1. Sayıları hesapla
mode_counts = df_selected_v2['mode'].value_counts().reset_index()
mode_counts.columns = ['mode', 'count']

# 2. Yüzdeleri hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

public_modes = [
    'Belediye Otobüsü', 'Metro', 'Minibüs', 'Metrobüs', 'Marmaray', 'Tramvay', 'Dolmuş',
    'Şehir Hatları Vapuru', 'Deniz Otobüsü (IDO)', 'Deniz Motoru', 
    'Halkalı-Bahçeşehir Raylı Sistem Hattı', 'Füniküler', 'Teleferik', 'Tünel', 'Deniz Taksi'
]
selected_modes = mode_counts[mode_counts['mode'].isin(public_modes)]
selected_modes

<span style="font-size:22pt; color:red;"><b>Figure 99. Public Transportation Mode choice (% distribution among all PT) (İstanbul SUMP Stage II HTS)</b></span>
### 
Bu tablo, toplu taşıma sistemine ait belirli modların mode kolonuna göre gruplandırılmasıyla oluşturulmuştur. Her bir mod için:


**count**: Bu modla yapılan toplam yolculuk sayısını,

**percentage**: Yalnızca tabloya dahil edilen 15 toplu taşıma modunun toplamı içinde bu modun oranını göstermektedir.

In [ ]:
# Toplam sadece selected_modes içindeki count'lardan alınır
total_selected = selected_modes['count'].sum()

# Yüzdelikleri yeniden hesapla
selected_modes['percentage'] = round((selected_modes['count'] / total_selected) * 100, 2)
selected_modes

<span style="font-size:22pt; color:red;"><b>Figure 100.Sea transportation mode choice (% distribution among all sea transport) (İstanbul SUMP Stage II HTS)</b></span>

### 

In [ ]:
# Sadece deniz ulaşımı modlarını seç
public_modes = ['Şehir Hatları Vapuru', 'Deniz Otobüsü (IDO)', 'Deniz Motoru', 'Deniz Taksi']
selected_modes = mode_counts[mode_counts['mode'].isin(public_modes)].copy()

# Yüzdelikleri yeniden hesapla (sadece bu 4 mod içindeki toplam baz alınır)
total_selected = selected_modes['count'].sum()
selected_modes['percentage'] = round((selected_modes['count'] / total_selected) * 100, 2)
selected_modes

<span style="font-size:22pt; color:red;"><b>Figure 101.Shared mobility mode choice (% distribution among all shared mobility) (İstanbul SUMP Stage II HTS)</b></span>
###  

In [ ]:
# Seçilecek modlar
micromobility_modes = ['Paylaşımlı Bisiklet', 'Paylaşımlı Skuter', 'Skuter', 'Bisiklet']

# İlgili modlara göre filtreleme
micromobility_df = df_selected_v2[df_selected_v2['mode'].isin(micromobility_modes)].copy()

# Sayı ve yüzde hesaplama
mode_counts = micromobility_df['mode'].value_counts().reset_index()
mode_counts.columns = ['mode', 'count']
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

mode_counts

 <span style="font-size:22pt; color:red;"><b>Figure 315. Daily Average Travel, Access, Egress, In-Vehicle & Waiting time across Public Transportation Modes (İstanbul SUMP Stage II HTS))</b></span>
 
 Bu bölümde, **toplu taşıma araçlarının türlerine göre** yolculuk süreleri ve bileşenleri analiz edilmiştir.

### 

- **Toplam Yolculuk Süresi (`travel_time`)**
- **Araçta Geçirilen Süre (`vehicle_time`)**
- **Ulaşıma Harcanan Süre (`access_time`)**
- **İnilen Durağın Varış Noktasına Süre (`egress_time`)**
- **Bekleme Süresi (`waiting_time`)**

In [ ]:
import pandas as pd

# Belirlenen toplu taşıma modları
pt_modes = [
    "Belediye Otobüsü",
    "Deniz Motoru",
    "Deniz Otobüsü (IDO)",
    "Deniz Taksi",
    "Dolmuş",
    "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Marmaray",
    "Metro",
    "Metrobüs",
    "Minibüs",
    "Teleferik",
    "Tramvay",
    "Şehir Hatları Vapuru"
]

# Belirli modlara göre filtreleme
filtered_df = df_selected_v2[df_selected_v2["mode"].isin(pt_modes)]

# Ortalama süreleri hesapla
pt_mean = filtered_df.groupby("mode")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().reset_index()

# Virgülden sonra 1 basamak
pt_mean = pt_mean.round(1)

# Genel ortalamayı hesapla ve ekle
overall = pt_mean.drop(columns="mode").mean(numeric_only=True).round(1)
overall["mode"] = "Genel Ortalama"
pt_mean = pd.concat([pt_mean, pd.DataFrame([overall])], ignore_index=True)

pt_mean

 <span style="font-size:22pt; color:red;"><b>Figure 154.Travel Time of Disabled People to Public Transport (İstanbul SUMP Stage II HTS)</b></span>

###  

In [ ]:
pro = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Profile.xlsx')
dis = pro[pro['disability'] == 'Evet'] 
df_selected_v3 = df_selected_v2

In [ ]:
dis['p_id'] = dis['p_id'].astype(str)
df_selected_v3['p_id'] = df_selected_v3['p_id'].astype(str)
print("dis p_id örnekleri:")
print(dis['p_id'].astype(str).str.strip().unique()[:10])

print("\ndf_selected_v2 p_id örnekleri:")
print(df_selected_v3['p_id'].astype(str).str.strip().unique()[:10])


In [ ]:
import re

dis['p_id_clean'] = dis['p_id'].astype(str).apply(lambda x: ''.join(re.findall(r'\d+', x)))
df_selected_v3['p_id_clean'] = df_selected_v3['p_id'].astype(str).apply(lambda x: ''.join(re.findall(r'\d+', x)))

matched_df = df_selected_v3[df_selected_v3['p_id_clean'].isin(dis['p_id_clean'])]

common_ids = set(df_selected_v3['p_id_clean']).intersection(set(dis['p_id_clean']))
print(f"Eşleşen p_id sayısı: {len(common_ids)}")


In [ ]:
disab_mean= matched_df.groupby("mode")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
    
]].mean().reset_index()
disab_mean

<span style="font-size:22pt; color:red;"><b>Figure 316-324 Public Transport Acces -egress time by hh-orig-dest district in İstanbul (İstanbul SUMP Stage II HTS) </b></span>

### 

🚏 Toplu Taşımada Yolculukların Başlangıç ve Bitiş Noktalarının Analizi

Bu bölümde, **yalnızca "Public Transport" (Toplu Taşıma)** modu içeren yolculuk grupları üzerinden detaylı bir analiz gerçekleştirilmiştir. Her `same_p_and_k_y` grubunun, yani aynı kişiye ait ve aynı gün içerisindeki toplu taşıma içeren yolculuklarının:
- **nerede başladığı** (ilk durak),
- **nerede bittiği** (son durak),
- ve bu duraklara erişim ve çıkış süreleri analiz edilmiştir.

 🔍 Analiz Kapsamı ve Yöntemi

1. **Veri Seçimi**  
   Tüm yolculuklar içerisinden yalnızca `mode_category` kolonu `"Public Transport"` olan satırlar seçilmiştir. Bu sayede, yürüyüş, özel araç gibi modlar dışarıda bırakılarak sadece toplu taşıma modunun analiz edilmesi sağlanmıştır.

2. **Başlangıç Bilgisi (Access Time)**
   - Her grup (`same_p_and_k_y`) için **en küçük ID**'ye sahip satır, yolculuğun başladığı adım olarak kabul edilmiştir.
   - Bu adımdan `access_time`, `orig_dist` (başlangıç ilçesi) ve `district` (hane ilçesi) bilgileri alınmıştır.

3. **Bitiş Bilgisi (Egress Time)**
   - Her grup için **en büyük ID**'ye sahip satır, yolculuğun sonlandığı adım olarak değerlendirilmiştir.
   - Bu adımdan `egress_time` ve `dest_dist` (varış ilçesi) bilgileri alınmıştır.

4. **Birleştirme**
   - Başlangıç ve bitiş bilgileri aynı grup (`same_p_and_k_y`) üzerinden birleştirilerek yolculukların erişim ve varış özetleri çıkarılmıştır.

📌 Elde Edilen Ana Göstergeler

- `hh_loc`: Katılımcının yaşadığı ilçe (hane lokasyonu)
- `first_orig`: Yolculuğun başladığı ilçe (genelde evden)
- `access_time_final`: Toplu taşımaya ulaşmak için yürünmesi gereken süre (dakika)
- `final_dest`: Toplu taşımanın sonlandığı ilçe
- `egress_time_final`: Toplu taşımadan indikten sonra son noktaya ulaşmak için geçen süre (dakika)



In [ ]:
# ID'leri sayıya çevir
df_selected_v2["id"] = df_selected_v2["id"].astype(str).str.replace(",", "").astype(int)

# 🎯 Public Transport içeren grupları bul (diğer modlar olabilir)
pt_groups = df_selected_v2[df_selected_v2["mode_category"] == "Public Transport"]["same_p_and_k_y"].unique()

# Sadece bu grupları al
df_filtered = df_selected_v2[df_selected_v2["same_p_and_k_y"].isin(pt_groups)]

# Sadece Public Transport satırlarını al (süreleri bu moddan alacağız)
df_pt_only = df_filtered[df_filtered["mode_category"] == "Public Transport"].copy()

# En küçük id'li Public Transport satırı (access_time için)
min_id_rows = df_pt_only.loc[
    df_pt_only.groupby("same_p_and_k_y")["id"].idxmin()
].copy()

min_id_rows["access_time_final"] = min_id_rows["access_time"]

initial_info = min_id_rows[[
    "same_p_and_k_y", "district", "p_id", "access_time_final", "orig_dist"
]].rename(columns={
    "district": "hh_loc",
    "orig_dist": "first_orig"
})

# En büyük id'li Public Transport satırı (egress_time ve varış için)
max_id_rows = df_pt_only.loc[
    df_pt_only.groupby("same_p_and_k_y")["id"].idxmax()
].copy()

max_id_rows["egress_time_final"] = max_id_rows["egress_time"]

final_info = max_id_rows[[
    "same_p_and_k_y", "dest_dist", "egress_time_final"
]].rename(columns={
    "dest_dist": "final_dest"
})

# Her şeyi birleştir
final_result = initial_info.merge(final_info, on="same_p_and_k_y", how="inner")


In [ ]:
orig = final_result.groupby("first_orig")[[
    "access_time_final",
    "egress_time_final",

]].mean().reset_index()
orig.head(2)


In [ ]:
hh = final_result.groupby("hh_loc")[[
    "access_time_final",
    "egress_time_final",

]].mean().reset_index()
hh.to_excel("hh.xlsx")

In [ ]:
dest = final_result.groupby("final_dest")[[
    "access_time_final",
    "egress_time_final",

]].mean().reset_index()
dest.head(2)
dest.to_excel("dest.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 416 -417-418 Average Travel Time of Walking Trips by Districts of İstanbul (İstanbul SUMP Stage II HTS)</b></span>

### 


In [ ]:
test = df_selected_v2.groupby(["same_p_and_k_y", "mode_category"], as_index=False).agg({
    "travel_time": "sum",
    "vehicle_time": "sum",
    "access_time": "sum",
    "egress_time": "sum",
    "waiting_time": "sum",
    "district": "first",
    "p_id": "first",
    "id": "first"
})
test

In [ ]:
import pandas as pd

# 1. Walking modunu filtrele
walking_df = test[test['mode_category'] == 'Walking'].copy()

# 2. Sadece İstanbul ilçelerini al
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]
walking_df = walking_df[walking_df["district"].isin(istanbul_ilceleri)]

# 3. İlçe bazlı ortalamaları hesapla
grouped_means_4 = walking_df.groupby("district")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().reset_index()

# 4. Genel ortalamayı hesapla ve ekle
genel_ortalama = walking_df[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().to_frame().T

genel_ortalama["district"] = "GENEL ORTALAMA"

# Sıralamayı koruyarak birleştir
grouped_means_4 = pd.concat([grouped_means_4, genel_ortalama], ignore_index=True)

grouped_means_4.head(1)


In [ ]:
import pandas as pd

# 1. Walking modunu filtrele
walking_df = test[test['mode_category'] == 'Walking'].copy()

# 2. Sadece İstanbul ilçelerini al
tekirdag_districts = [
    'ÇERKEZKÖY', 'ÇORLU', 'ERGENE', 'KAPAKLI', 'MARMARAEREĞLİSİ', 'SARAY'
]
walking_df = walking_df[walking_df["district"].isin(tekirdag_districts)]

# 3. İlçe bazlı ortalamaları hesapla
grouped_means_4 = walking_df.groupby("district")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().reset_index()

# 4. Genel ortalamayı hesapla ve ekle
genel_ortalama = walking_df[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().to_frame().T

genel_ortalama["district"] = "GENEL ORTALAMA"

# Sıralamayı koruyarak birleştir
grouped_means_4 = pd.concat([grouped_means_4, genel_ortalama], ignore_index=True)

grouped_means_4

In [ ]:
import pandas as pd

# 1. Walking modunu filtrele
walking_df = test[test['mode_category'] == 'Walking'].copy()

# 2. Sadece İstanbul ilçelerini al
kocaeli_districts = ['ÇAYIROVA', 'DARICA', 'GEBZE']
walking_df = walking_df[walking_df["district"].isin(kocaeli_districts)]

# 3. İlçe bazlı ortalamaları hesapla
grouped_means_4 = walking_df.groupby("district")[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().reset_index()

# 4. Genel ortalamayı hesapla ve ekle
genel_ortalama = walking_df[[
    "travel_time",
    "vehicle_time",
    "access_time",
    "egress_time",
    "waiting_time"
]].mean().to_frame().T

genel_ortalama["district"] = "GENEL ORTALAMA"

# Sıralamayı koruyarak birleştir
grouped_means_4 = pd.concat([grouped_means_4, genel_ortalama], ignore_index=True)

grouped_means_4.head(1)


<span style="font-size:22pt; color:red;"><b>Table 60. Average Transfer Durations Between Different Public Transportation Modes (in minutes) (İstanbul SUMP Stage II HTS)</b></span>

### 
Toplu Taşımada Modlar Arası Transfer Süresi Analizi

Bu bölümde, **sadece "Public Transport" (Toplu Taşıma)** modunu tercih eden yolculuk grupları üzerinden modlar arası geçiş süreleri analiz edilmiştir. Amaç, İstanbul’da toplu taşıma kullanıcılarının bir taşıma türünden diğerine geçerken karşılaştıkları ortalama bekleme veya yürüme sürelerini ortaya koymaktır.

-
#### 📊 Analiz Kapsamı ve Yöntemi

1. **Veri Filtreleme**  
   Analize sadece `"mode_choice"` değeri `"Public Transport"` olan yolculuklar dahil edilmiştir. Ayrıca `"Servis"`, `"Yürüme"`, `"Taksi"` gibi analiz dışı modlar da hariç bırakseçilmiştir.

3. **İstenmeyen Modların Hariç Tutulması**  
   Herhangi bir adımında özel ya da bireysel taşıma içeren modlara sahip olan yolculuk grupları tamamen dışlanmıştır.

4. **Transfer Süre   - Hesaplama**  
   Her yolculuk grubu içinde, bir moddan diğerine geçilen her adım için:
   - İlk moddan inme süresi (`egress_time`)
   - İkinci moda binme bilgisiyle eşleştirilmiş
   - İlgili iki mod arasındaki transfer süresi hesaplanmıştır.

5. **Sonuçların Matris Haline Getirilmesi**  
   Elde edilen süreler, modlar arası transfer matrisine işlenmiş ve ortalamaları alınmıştır. Matrisin satırları **başlangıç modu**, sütunları ise **varışkı sağlamak amacıyla hazırlanmıştır.



In [ ]:
import pandas as pd

# Hariç tutulacak modlar listesi
excluded_modes = [
    "Diğer", "Yürüme", "Özel Araç", "Servis", "Motosiklet", "Taksi",
    "Skuter", "Kurum/ Firma Aracı", "Paylaşımlı Araç", "Paylaşımlı Skuter", "Bisiklet"
]

# 1. Sadece Public Transport olarak seçilenleri al
df_filter = df_selected_v2[df_selected_v2["mode_choice"] == "Public Transport"].copy()

# 2. Birden fazla satıra sahip same_p_and_k_y’leri al
repeated_keys = df_filter["same_p_and_k_y"].value_counts()
repeated_keys = repeated_keys[repeated_keys > 1].index
df_repeated = df_filter[df_filter["same_p_and_k_y"].isin(repeated_keys)].copy()

# 3. İçinde excluded_modes olan same_p_and_k_y’leri belirle
has_excluded = df_repeated[df_repeated["mode"].isin(excluded_modes)]["same_p_and_k_y"].unique()

# 4. Bu grupları hariç tut
df_cleaned = df_repeated[~df_repeated["same_p_and_k_y"].isin(has_excluded)].copy()


In [ ]:
import pandas as pd
import numpy as np

# Mode'lar listesini al
modes = df_cleaned['mode'].unique()
result_matrix = pd.DataFrame(index=modes, columns=modes)

# Her yolculuk grubu için transfer sürelerini matrise yaz
for group_id, group in df_cleaned.groupby('same_p_and_k_y'):
    group = group.reset_index(drop=True)
    for i in range(len(group) - 1):  # son satır hariç
        from_mode = group.loc[i, 'mode']
        to_mode = group.loc[i + 1, 'mode']
        transfer_time = group.loc[i, 'egress_time']

        # Hücrede liste yoksa başlat
        current_val = result_matrix.loc[from_mode, to_mode]
        if not isinstance(current_val, list):
            result_matrix.loc[from_mode, to_mode] = []

        # Transfer süresini listeye ekle
        result_matrix.loc[from_mode, to_mode].append(transfer_time)

# Listeleri ortalamaya çevir
for row in result_matrix.index:
    for col in result_matrix.columns:
        val = result_matrix.loc[row, col]
        if isinstance(val, list) and val:
            result_matrix.loc[row, col] = round(sum(val) / len(val), 1)
        else:
            result_matrix.loc[row, col] = ''

# Tüm hücreleri sayıya çevir (boşları NaN yap)
result_numeric = result_matrix.replace('', np.nan).astype(float)

# Satır (row) ortalamaları
row_means = result_numeric.mean(axis=1).round(1)
result_matrix['Ortalama'] = row_means

# Sütun (column) ortalamaları
col_means = result_numeric.mean(axis=0).round(1)
col_means['Ortalama'] = round(row_means.mean(), 1)  # Genel ortalama

# En alta ortalama satırını ekle
result_matrix.loc['Ortalama'] = col_means

result_matrix

<span style="font-size:22pt; color:red;"><b>Figure 307. Mode Choice Public Transport by District in İstanbul Based on Trip Legs (İstanbul SUMP Stage II HTS)</b></span>

### 

In [ ]:
import pandas as pd

# Ulaşım modu grupları
rubber_tired = [
    "Belediye Otobüsü", "Dolmuş", "Metrobüs", "Minibüs"
]

rail_transport = [
    "Metro", "Marmaray", "Tramvay", "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Füniküler", "Tünel", "Teleferik"
]

sea_transport = [
    "Deniz Otobüsü (IDO)", "Şehir Hatları Vapuru", "Deniz Motoru", "Deniz Taksi"
]

# Mod gruplama fonksiyonu
def map_transport_mode(mode):
    if mode in rubber_tired:
        return "Rubber-Tired Transport"
    elif mode in rail_transport:
        return "Rail Transport"
    elif mode in sea_transport:
        return "Sea Transport"
    else:
        return "Other"

# Yeni DataFrame oluştur, grubu ekle
df_grouped_v2 = df_selected_v2[["district", "mode"]].copy()
df_grouped_v2["mode_group"] = df_grouped_v2["mode"].apply(map_transport_mode)

# "Other" kategorisini çıkar
df_grouped_v2 = df_grouped_v2[df_grouped_v2["mode_group"] != "Other"]

# Pivot tablo oluştur
pivot_table = df_grouped_v2.pivot_table(
    index="district",
    columns="mode_group",
    aggfunc="size",
    fill_value=0
).reset_index()

# Sonuç
pivot_table.head(1)


<span style="font-size:22pt; color:red;"><b>Figure 310. Mode Choice by Travel Purpose Based on Trip Legs (İstanbul SUMP Stage II HTS) </b></span>

### 

In [ ]:
import pandas as pd

# 1. PT modlarını içeren liste
pt_modes = [
    "Belediye Otobüsü",
    "Deniz Motoru",
    "Deniz Otobüsü (IDO)",
    "Deniz Taksi",
    "Dolmuş",
    "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Marmaray",
    "Metro",
    "Metrobüs",
    "Minibüs",
    "Teleferik",
    "Tramvay",
    "Şehir Hatları Vapuru"
]

# 2. Amaç eşleme fonksiyonu
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Filtreleme
filtered_df = df_selected_v2[
    df_selected_v2['mode'].isin(pt_modes) & 
    (df_selected_v2['purpose'] != 'Eve gidiş')
].copy()

# 4. map_purpose ile gruplandırılmış amaç sütununu ekle (geçici)
filtered_df['purpose_group'] = filtered_df['purpose'].map(map_purpose)

# 5. Mode ve grouped purpose bazında sayım
pivot_result = (
    filtered_df
    .groupby(['mode', 'purpose_group'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 6. Genel toplam satırı ekle
total_row = pivot_result.drop(columns='mode').sum(numeric_only=True)
total_row['mode'] = 'TOPLAM'

# 7. Sonuç birleştirme
final_result = pd.concat([pivot_result, pd.DataFrame([total_row])], ignore_index=True)
final_result.head(2)
final_result.to_excel("purpose.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 311. Mode Choice of Public Transport among Work Based on Trip Legs (İstanbul SUMP Stage II HTS)</b></span>

### 

In [ ]:
import pandas as pd

# 1. PT modlarını içeren liste
pt_modes = [
    "Belediye Otobüsü",
    "Deniz Motoru",
    "Deniz Otobüsü (IDO)",
    "Deniz Taksi",
    "Dolmuş",
    "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Marmaray",
    "Metro",
    "Metrobüs",
    "Minibüs",
    "Teleferik",
    "Tramvay",
    "Şehir Hatları Vapuru"
]

# 2. Amaç eşleme fonksiyonu
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Filtreleme (sadece pt_modes ve Eve gidiş hariç)
filtered_df = df_selected_v2[
    df_selected_v2['mode'].isin(pt_modes) & 
    (df_selected_v2['purpose'] != 'Eve gidiş')
].copy()

# 4. map_purpose ile amaç eşleme
filtered_df['purpose_group'] = filtered_df['purpose'].map(map_purpose)

# 5. Sadece 'Work' olanları filtrele
work_df = filtered_df[filtered_df['purpose_group'] == 'Work']

# 6. mode bazında Work sayısı
work_counts = (
    work_df
    .groupby('mode')
    .size()
    .reset_index(name='Work')
)

# 7. Genel toplam satırı ekle
total_row = pd.DataFrame([{
    'mode': 'TOPLAM',
    'Work': work_counts['Work'].sum()
}])

# 8. Sonuçları birleştir
final_result = pd.concat([work_counts, total_row], ignore_index=True)

# 9. İlk 2 satırı göster
final_result.head(2)
final_result.to_excel("work.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 313. Mode choice of Public Transportation Based on Person Types Based on Trip Legs (İstanbul SUMP Stage II HTS)</b></span>

### 

In [ ]:
import pandas as pd

# PT modlarını içeren liste
pt_modes = [
    "Belediye Otobüsü",
    "Deniz Motoru",
    "Deniz Otobüsü (IDO)",
    "Deniz Taksi",
    "Dolmuş",
    "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Marmaray",
    "Metro",
    "Metrobüs",
    "Minibüs",
    "Teleferik",
    "Tramvay",
    "Şehir Hatları Vapuru"
]

# Yalnızca PT modlarını içeren satırları filtrele
df_pt_only = df_selected_v2[df_selected_v2['mode'].isin(pt_modes)]

# mode ve Employment Group'a göre sayım yap
employment_counts_by_mode = (
    df_pt_only.groupby(['mode', 'Employment Group'])
    .size()
    .reset_index(name='count')
)

# Pivot işlemi: her mode bir kolon, Employment Group satır olur
employment_counts_pivot = employment_counts_by_mode.pivot(
    index='Employment Group',
    columns='mode',
    values='count'
)

# Eksik (NaN) değerleri 0 ile doldur ve tam sayı yap
employment_counts_pivot = employment_counts_pivot.fillna(0).astype(int)

# Kolon sırasını pt_modes listesine göre düzenle (opsiyonel)
employment_counts_pivot = employment_counts_pivot.reindex(columns=pt_modes)

# İstersen Employment Group'u tekrar kolona çevirebilirsin
employment_counts_pivot = employment_counts_pivot.reset_index()

# Sonuç
employment_counts_pivot.to_excel("employ.xlsx")


In [ ]:
employment_counts_pivot

<span style="font-size:22pt; color:red;"><b>Figure 313. Mode choice of Public Transportation Based on Person Types Based on Trip Legs (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, bireylerin ait olduğu **hane gelir grubuna (income_group)** göre hangi **ulaşım modlarını (mode_choice)** tercih ettiklerini göstermektedir. Hanehalkı gelir bilgileri, belirli aralıklarla tanımlanmış ve üç ana gruba ayrılmıştır:

#### Gelir Grupları:
- **Low**: 25.000 TL altı (düşük gelirli)
- **Medium**: 25.001 – 80.000 TL arası (orta gelirli)
- **High**: 80.001 TL ve üzeri (yüksek gelirli)
- **Not Specified**: Gelir bilgisi olmayanlar



In [ ]:
house = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Household.xlsx')

In [ ]:
import pandas as pd

# 1. df_selected_v2 ve house tablosunu hh_id üzerinden birleştir
merged = df_selected_v2.merge(house, on='hh_id', how='left')

# 2. Gelir grubunu sınıflandıran fonksiyon
def categorize_income(income):
    high = ['100.000TL’den fazla', '100.001 - 140.000 TL', '140.001 - 200.000 TL',
            '200.001TL’den fazla', '60.001 - 100.000 TL', '80.001 - 100.000 TL']
    medium = ['25.001 - 40.000 TL', '40.001 - 60.000 TL', '60.001 - 80.000 TL']
    low = ['5.000 TL’den az', '5.001 – 10.000 TL', '10.001 - 15.000 TL', '15.001 - 25.000 TL']
    
    if income in high:
        return 'High'
    elif income in medium:
        return 'Medium'
    elif income in low:
        return 'Low'
    else:
        return 'Not Specified'

# 3. Gelir grubu sütununu oluştur
merged['income_group'] = merged['hh_income'].apply(categorize_income)

# 4. Toplu taşıma modlarını tanımla
pt_modes = [
    "Belediye Otobüsü",
    "Deniz Motoru",
    "Deniz Otobüsü (IDO)",
    "Deniz Taksi",
    "Dolmuş",
    "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Marmaray",
    "Metro",
    "Metrobüs",
    "Minibüs",
    "Teleferik",
    "Tramvay",
    "Şehir Hatları Vapuru"
]

# 5. Yalnızca PT modlarını filtrele
df_pt_only = merged[merged['mode'].isin(pt_modes)]

# 6. mode ve income_group'a göre sayım
income_counts_by_mode = (
    df_pt_only.groupby(['mode', 'income_group'])
    .size()
    .reset_index(name='count')
)

# 7. Pivot: income_group'lar satır, mode'lar sütun
income_counts_pivot = income_counts_by_mode.pivot(
    index='income_group',
    columns='mode',
    values='count'
)

# 8. Eksik değerleri 0 ile doldur ve tam sayıya çevir
income_counts_pivot = income_counts_pivot.fillna(0).astype(int)

# 9. Kolon sırasını pt_modes listesine göre düzenle
income_counts_pivot = income_counts_pivot.reindex(columns=pt_modes)

# 10. İstersen index'i sıfırla
income_counts_pivot = income_counts_pivot.reset_index()

income_counts_pivot

<span style="font-size:22pt; color:red;"><b>Figure 315. Mode choice of Public Transportation Based on Car Ownership Based on Trip Legs (İstanbul SUMP Stage II HTS)</b></span>

####


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
vehicle = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Vehicle_ownership.xlsx') 

In [ ]:
vehicle = vehicle[['hh_id', 'most_user']]

In [ ]:
vehicle.head(1)

In [ ]:
vehicle['person_id'] = vehicle['hh_id'] + '-' + vehicle['most_user']

In [ ]:
vehicle_unique = vehicle.drop_duplicates(subset=['person_id'])
merged_df = df_selected_v2.merge(vehicle_unique, on='person_id', how='left', indicator=True)
merged_df['ownership_status'] = merged_df['_merge'].apply(lambda x: 'yes' if x == 'both' else 'no')
merged_df.drop(columns=['_merge'], inplace=True)
merged_df.head(1)

In [ ]:
import pandas as pd

# pt_modes listesi
pt_modes = [
    "Belediye Otobüsü",
    "Deniz Motoru",
    "Deniz Otobüsü (IDO)",
    "Deniz Taksi",
    "Dolmuş",
    "Füniküler",
    "Halkalı-Bahçeşehir Raylı Sistem Hattı",
    "Marmaray",
    "Metro",
    "Metrobüs",
    "Minibüs",
    "Teleferik",
    "Tramvay",
    "Şehir Hatları Vapuru"
]

# 1. pt_modes'a göre filtrele
filtered = merged_df[merged_df['mode'].isin(pt_modes)]

# 2. ownership_status ve mode'a göre saydır
count_df = filtered.groupby(['ownership_status', 'mode']).size().unstack(fill_value=0)




<span style="font-size:22pt; color:red;"><b>Figure 332-33-334. The Temporal Usage Patterns of Municipal Buses, Dolmuş, Metrobus, and Minibuses</b></span>

####


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# 1. PT modları listesi
pt_modes = [
    "Belediye Otobüsü", "Deniz Motoru", "Deniz Otobüsü (IDO)", "Deniz Taksi",
    "Dolmuş", "Füniküler", "Halkalı-Bahçeşehir Raylı Sistem Hattı", "Marmaray",
    "Metro", "Metrobüs", "Minibüs", "Teleferik", "Tramvay", "Şehir Hatları Vapuru"
]

# 2. Veriyi filtrele (df_selected_v2 daha önceden tanımlanmış olmalı)
df_pt = df_selected_v2[df_selected_v2['mode'].isin(pt_modes)].copy()

# 3. start_time string'ini saat formatına çevir
df_pt['start_time_clean'] = pd.to_datetime(df_pt['start_time'], format="%H:%M", errors='coerce').dt.time

# 4. 30 dakikalık zaman aralıklarını oluştur (00:00 - 00:00 → ertesi gün 00:00 dahil)
def generate_time_bins(start="00:00", end="00:00", freq=30):
    bins = []
    start_dt = datetime.strptime(start, "%H:%M")
    end_dt = datetime.strptime(end, "%H:%M")
    if end_dt <= start_dt:
        end_dt += timedelta(days=1)  # 00:00 ertesi gün olarak kabul edilir
    while start_dt < end_dt:
        next_dt = start_dt + timedelta(minutes=freq)
        bins.append((start_dt.time(), next_dt.time()))
        start_dt = next_dt
    return bins

# 5. Zaman aralığı etiketlerini oluştur
time_bins = generate_time_bins()
labels = [f"{s.strftime('%H:%M')} - {e.strftime('%H:%M')}" for s, e in time_bins]

# 6. Her satırın hangi zaman aralığına denk geldiğini hesapla
def assign_time_bin(t):
    if pd.isna(t):
        return np.nan
    for i, (start, end) in enumerate(time_bins):
        if start <= t < end or (start > end and (t >= start or t < end)):
            return labels[i]
    return np.nan

df_pt['time_range'] = df_pt['start_time_clean'].apply(assign_time_bin)

# 7. Pivot tablo: modlar satırda, zaman aralıkları sütunda
pivot_table = (
    df_pt
    .groupby(['mode', 'time_range'])
    .size()
    .unstack(fill_value=0)
    .reindex(index=pt_modes, columns=labels, fill_value=0)
)




<span style="font-size:22pt; color:red;"><b>Figure 312. Students’ Mode Choice of Public Transport for School Trips Based on Trip Legs (İstanbul SUMP Stage II HTS)</b></span>

####


In [ ]:
import pandas as pd

# 1. Toplu taşıma modlarını tanımla
pt_modes = [
    "Belediye Otobüsü", "Deniz Motoru", "Deniz Otobüsü (IDO)", "Deniz Taksi",
    "Dolmuş", "Füniküler", "Halkalı-Bahçeşehir Raylı Sistem Hattı", "Marmaray",
    "Metro", "Metrobüs", "Minibüs", "Teleferik", "Tramvay", "Şehir Hatları Vapuru"
]

# 2. Türkçe - İngilizce öğrenci tanımı eşleştirme sözlüğü
edu_translation = {
    "Açıköğretim lise öğrenci": "Open High School Student",
    "Açıköğretim ortaöğretim öğrenci": "Open Secondary School Student",
    "Açıköğretim üniversite öğrenci": "Open University Student",
    "Kreş/Okul öncesi öğrenci": "Preschool/Kindergarten Student",
    "Lise/ Dengi öğrenci": "High School or Equivalent Student",
    "Ortaöğretim öğrenci": "Secondary School Student",
    "Y.Lisans/Doktora öğrenci": "Graduate Student (Master’s/PhD)",
    "Üniversite öğrenci (Ön Lisans/Lisans)": "University Student (Associate/Bachelor’s Degree)",
    "İlköğretim öğrenci": "Primary School Student"
}

# 3. Sadece istenen öğrenci türlerini filtrele
filtered_students = df_selected_v2[df_selected_v2["edu"].isin(edu_translation.keys())].copy()

# 4. İngilizce karşılıkları yeni kolona yaz
filtered_students["edu_en"] = filtered_students["edu"].map(edu_translation)

# 5. Toplu taşıma modlarını filtrele
filtered_students = filtered_students[filtered_students["mode"].isin(pt_modes)]

# 6. Sayım: edu_en + mode kırılımında grup ve pivot
pt_counts_by_edu = (
    filtered_students
    .groupby(["edu_en", "mode"])
    .size()
    .unstack(fill_value=0)
    .reindex(columns=pt_modes, fill_value=0)
    .reset_index()
)

pt_counts_by_edu

In [ ]:
<span style="font-size:22pt; color:red;"><b>Figure 404. Average Hourly Temporal Micromobility Movement Pattern based on Trip Leg Analysis (İstanbul SUMP Stage II HTS)</b></span>

####


In [ ]:
import pandas as pd

# Filtrelenecek modlar
modes = ["Bisiklet", "Skuter", "Paylaşımlı Bisiklet", "Paylaşımlı Skuter"]

# İlgili modları filtrele
filtered_df = df_selected_v2[df_selected_v2["mode"].isin(modes)].copy()

# Paylaşımlı modları grupla
def group_mode(mode):
    if mode in ["Paylaşımlı Bisiklet", "Paylaşımlı Skuter"]:
        return "Paylaşımlı Mikromobilite"
    return mode

filtered_df["mode_grouped"] = filtered_df["mode"].apply(group_mode)

# start_time saat dilimine göre saatlik gruplama
filtered_df["hour"] = pd.to_datetime(filtered_df["start_time"]).dt.hour

# Saat aralıklarını belirle
bins = list(range(0, 25))  # 0-24
labels = [f"{str(i).zfill(2)}:00-{str((i+1)%24).zfill(2)}:00" for i in range(24)]
filtered_df["hour_range"] = pd.cut(filtered_df["hour"], bins=bins, labels=labels, right=False, include_lowest=True)

# Gruplayarak say
result = filtered_df.groupby(["hour_range", "mode_grouped"]).size().unstack(fill_value=0)

# Sonuçları saat sırasına göre sırala
result = result.reindex(labels)

result.head(2)


<span style="font-size:22pt; color:red;"><b>Figure 404. Average Hourly Temporal Micromobility Movement Pattern based on Trip Leg Analysis (İstanbul SUMP Stage II HTS)</b></span>



####  **Tablo Açıklaması**:


In [ ]:
import pandas as pd

# Filtrelenecek modlar
modes = ["Bisiklet", "Skuter", "Paylaşımlı Bisiklet", "Paylaşımlı Skuter"]

# İlgili modları filtrele
filtered_df = df_selected_v2[df_selected_v2["mode"].isin(modes)].copy()

# Paylaşımlı modları grupla
def group_mode(mode):
    if mode in ["Paylaşımlı Bisiklet", "Paylaşımlı Skuter"]:
        return "Paylaşımlı Mikromobilite"
    return mode

filtered_df["mode_grouped"] = filtered_df["mode"].apply(group_mode)

# start_time saat dilimine göre saatlik gruplama
filtered_df["hour"] = pd.to_datetime(filtered_df["start_time"]).dt.hour

# Saat aralıklarını belirle
bins = list(range(0, 25))  # 0-24
labels = [f"{str(i).zfill(2)}:00-{str((i+1)%24).zfill(2)}:00" for i in range(24)]
filtered_df["hour_range"] = pd.cut(filtered_df["hour"], bins=bins, labels=labels, right=False, include_lowest=True)

# Gruplayarak say
result = filtered_df.groupby(["hour_range", "mode_grouped"]).size().unstack(fill_value=0)

# Sonuçları saat sırasına göre sırala
result = result.reindex(labels)
result.head(1)


<span style="font-size:22pt; color:red;"><b>Figure 407. Demographic Patterns of Pedestrians based on Trip Leg Analysis (İstanbul SUMP Stage II HTS)</b></span>



####  **Tablo Açıklaması**:


In [ ]:
# 'Yürüme' modunu filtrele
walking_df = df_selected_v2[df_selected_v2["mode"] == "Yürüme"]

walking_by_employment_gender = (
    walking_df
    .groupby(["Employment Group", "gender"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)

pivot_table = walking_by_employment_gender.pivot(index="Employment Group", columns="gender", values="count").fillna(0)
pivot_table

<span style="font-size:22pt; color:red;"><b>Figure 408. Demographic Patterns of Cycling (İstanbul SUMP Stage II HTS) </b></span>



####  **Tablo Açıklaması**:


In [ ]:
cycling_df = df_selected_v2[df_selected_v2["mode"] == "Bisiklet"]

# 2. Employment Group ve gender'a göre gruplandır ve say
cycling_by_employment_gender = (
    cycling_df
    .groupby(["Employment Group", "gender"])
    .size()
    .reset_index(name="count")
)

# 3. Pivot tablo oluştur (satırlar: Employment Group, sütunlar: gender)
pivot_table = cycling_by_employment_gender.pivot(
    index="Employment Group",
    columns="gender",
    values="count"
).fillna(0)
pivot_table

<span style="font-size:22pt; color:red;"><b>Figure 266. Trip Leg P&R Trip Start Times (İstanbul SUMP Stage II HTS)</b></span>

In [ ]:
import pandas as pd

# Örnek veri — bunu kendi df_selected_v2 verinizle değiştirin
# df_selected_v2 = pd.read_csv("your_data.csv") gibi yüklenebilir

# 1. Ulaşım mod gruplarını tanımla
private_modes = ['Özel Araç', 'Paylaşımlı Araç', 'Motosiklet', 'Taksi', 'Diğer', 'Korsan Taksi','Kurum/ Firma Aracı']
pt_modes = ['Belediye Otobüsü', 'Deniz Motoru', 'Deniz Otobüsü (IDO)', 'Deniz Taksi Dolmuş',
            'Füniküler', 'Halkalı-Bahçeşehir Raylı Sistem Hattı','Marmaray',
            'Metro', 'Metrobüs', 'Minibüs', 'Şehir Hatları Vapuru', 'Teleferik', 'Tramvay', 'Tünel']

# 2. Her same_p_and_k_y grubunun mode listesini al
mode_groups = df_selected_v2.groupby('same_p_and_k_y')['mode'].apply(list).reset_index()

# 3. Hem özel hem toplu taşıma içeren grupları filtrele
def contains_both_mode_types(modes):
    return any(m in private_modes for m in modes) and any(m in pt_modes for m in modes)

valid_keys = mode_groups[mode_groups['mode'].apply(contains_both_mode_types)]['same_p_and_k_y'].tolist()

# 4. Bu gruplara ait satırları filtrele
result_df = df_selected_v2[df_selected_v2['same_p_and_k_y'].isin(valid_keys)].copy()



In [ ]:
transition_data = []

# same_p_and_k_y bazında gruplama
for group_id, group in result_df.groupby("same_p_and_k_y"):
    group = group.sort_values("mode_order").reset_index(drop=True)  # sıralama önemli
    for i in range(len(group) - 1):
        mode1 = group.loc[i, "mode"]
        mode2 = group.loc[i + 1, "mode"]
        purpose = group.loc[i + 1, "purpose"]  # geçilen modun amacı
        start_time = group.loc[i, "start_time"]  # önceki modun başlama zamanı

        # Sadece özel araç ↔ toplu taşıma geçişlerini filtrele
        if (mode1 in private_modes and mode2 in pt_modes) or (mode1 in pt_modes and mode2 in private_modes):
            transition_data.append({
                "same_p_and_k_y": group_id,
                "mode_transition": f"{mode1}-{mode2}",
                "purpose": purpose,
                "start_time": start_time
            })

# Yeni DataFrame oluştur
transition_df = pd.DataFrame(transition_data)

In [ ]:
transition_df

In [ ]:
# 1. Saat bilgisi çıkar
transition_df['hour'] = pd.to_datetime(transition_df['start_time'], format='%H:%M').dt.hour

# 2. 24 saatlik zaman aralıkları oluştur
bins = list(range(0, 25))  # 0-24
labels = [f"{str(h).zfill(2)}:00-{str(h+1).zfill(2)}:00" for h in range(24)]

# 3. Zaman aralığına göre gruplandır
transition_df['time_group'] = pd.cut(
    transition_df['hour'], 
    bins=bins, 
    labels=labels, 
    right=False, 
    include_lowest=True
)

# 4. time_group dağılımını hesapla
time_group_distribution = transition_df['time_group'].value_counts().sort_index()

# 5. DataFrame formatında göster
time_group_df = time_group_distribution.reset_index()
time_group_df.columns = ['time_group', 'count']
time_group_df.to_excel("time.xlsx")

In [ ]:
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

filtered = transition_df[transition_df['purpose'] != 'Eve gidiş'].copy()
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

grouped = filtered[filtered['purpose_group'].notna()] \
    .groupby('purpose_group') \
    .size() \
    .reset_index(name='trip_count')

grouped['percentage'] = round(grouped['trip_count'] * 100 / grouped['trip_count'].sum(), 2)

grouped

<span style="font-size:22pt; color:red;"><b>Figure 267. Trip Leg: Transition from Private Vehicle to Public Transportation (İstanbul SUMP Stage II HTS)</b></span>

In [ ]:
# mode_transition kolonunu ayır
transition_df[['First_Mode', 'Next_Mode']] = transition_df['mode_transition'].str.split('-', expand=True)
transition_df.head(1)

In [ ]:
# Ayırmak istediğin modlar
private_modes = ['Özel Araç', 'Paylaşımlı Araç', 'Motosiklet', 'Taksi', 'Diğer', 'Korsan Taksi','Kurum/ Firma Aracı']

# İki ayrı DataFrame oluştur
first_mode_private_df = transition_df[transition_df['First_Mode'].isin(private_modes)].copy()
first_mode_public_df = transition_df[~transition_df['First_Mode'].isin(private_modes)].copy()



In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Örnek veri (senin df'in bununla aynı yapıya sahip olmalı)
# first_mode_public_df = pd.read_csv("your_data.csv")

# Benzersiz tüm modları bul
all_modes = pd.unique(first_mode_public_df[['First_Mode', 'Next_Mode']].values.ravel())

# Modları sayısal indekslere eşle
mode_to_index = {mode: i for i, mode in enumerate(all_modes)}

# Geçişleri say (First_Mode → Next_Mode)
mode_counts = (
    first_mode_public_df
    .groupby(['First_Mode', 'Next_Mode'])
    .size()
    .reset_index(name='count')
)

# Sankey diyagramı için kaynak, hedef ve değer sütunlarını hazırla
source_indices = mode_counts['First_Mode'].map(mode_to_index)
target_indices = mode_counts['Next_Mode'].map(mode_to_index)
values = mode_counts['count']

# Sankey diyagramı oluştur
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(mode_to_index.keys())
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="First Mode to Next Mode Sankey Diagram", font_size=12)
fig.show()


In [ ]:
mode_counts.to_excel("PT_Sankey.xlsx")

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Örnek veri (senin df'in bununla aynı yapıya sahip olmalı)
# first_mode_public_df = pd.read_csv("your_data.csv")

# Benzersiz tüm modları bul
all_modes = pd.unique(first_mode_private_df[['First_Mode', 'Next_Mode']].values.ravel())

# Modları sayısal indekslere eşle
mode_to_index = {mode: i for i, mode in enumerate(all_modes)}

# Geçişleri say (First_Mode → Next_Mode)
mode_counts = (
    first_mode_public_df
    .groupby(['First_Mode', 'Next_Mode'])
    .size()
    .reset_index(name='count')
)

# Sankey diyagramı için kaynak, hedef ve değer sütunlarını hazırla
source_indices = mode_counts['First_Mode'].map(mode_to_index)
target_indices = mode_counts['Next_Mode'].map(mode_to_index)
values = mode_counts['count']

# Sankey diyagramı oluştur
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(mode_to_index.keys())
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="First Mode to Next Mode Sankey Diagram", font_size=12)
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Benzersiz tüm modları bul (First_Mode ve Next_Mode içindeki tüm eşsiz değerler)
all_modes = pd.unique(first_mode_private_df[['First_Mode', 'Next_Mode']].values.ravel())

# Modları sayısal indekslere eşle
mode_to_index = {mode: i for i, mode in enumerate(all_modes)}

# Geçişleri say (First_Mode → Next_Mode)
mode_counts = (
    first_mode_private_df
    .groupby(['First_Mode', 'Next_Mode'])
    .size()
    .reset_index(name='count')
)

# Sankey diyagramı için kaynak, hedef ve değer sütunlarını hazırla
source_indices = mode_counts['First_Mode'].map(mode_to_index)
target_indices = mode_counts['Next_Mode'].map(mode_to_index)
values = mode_counts['count']

# Sankey diyagramı oluştur
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(mode_to_index.keys())
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="First Mode to Next Mode Sankey Diagram", font_size=12)
fig.show()
